In [1602]:
import numpy as np
import pandas as pd

# Load data

In [1603]:
# Load data
admissions = pd.read_csv('mimic_iii_data/ADMISSIONS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)

patients   = pd.read_csv('mimic_iii_data/PATIENTS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)

dx         = pd.read_csv('mimic_iii_data/DIAGNOSES_ICD.csv.gz',compression='gzip').drop('ROW_ID',axis=1)

drg        = pd.read_csv('mimic_iii_data/DRGCODES.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
drg.drop_duplicates(inplace=True)
drg.DESCRIPTION = drg.DESCRIPTION.map(lambda x: x.lower() if pd.notna(x) else x)

# Create DF

In [1604]:
# Create DF
X = admissions[['HADM_ID','ADMISSION_TYPE','ADMISSION_LOCATION','LANGUAGE','ETHNICITY','DIAGNOSIS']].copy()

### Add LOS

In [1605]:
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME)
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME)
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME)

admissions['LOS'] = (admissions.DISCHTIME - admissions.ADMITTIME).dt.days
# some discharge times are earlier than admission -- all seem to have presented DOA
print(admissions[admissions.LOS<0].sample(20))
X['LOS'] = admissions.LOS.map(lambda x: 0 if x<0 else x)
# Add column of pts who died within 4 hours of admission
X['DOA'] = ((admissions.DEATHTIME - admissions.ADMITTIME).dt.total_seconds()/3600<4).astype(int)

       SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME  \
32903       28945   116984 2123-07-11 15:34:00 2123-07-11 12:00:00   
10640        9155   179303 2179-12-28 12:30:00 2179-12-28 12:00:00   
37010       28760   168303 2167-01-14 13:29:00 2167-01-14 12:00:00   
3622         2858   190088 2108-09-25 15:29:00 2108-09-25 12:00:00   
20426       18567   143379 2133-02-08 15:24:00 2133-02-08 07:17:00   
56639       92001   150299 2191-09-25 05:08:00 2191-09-25 02:20:00   
45300       59551   128700 2191-11-01 13:00:00 2191-11-01 02:11:00   
58884       95367   139266 2135-04-03 14:16:00 2135-04-03 12:00:00   
39649       29761   184015 2143-01-24 16:31:00 2143-01-24 12:00:00   
4604         3915   198555 2178-04-07 16:31:00 2178-04-07 12:00:00   
34044       26947   135243 2154-10-06 13:28:00 2154-10-06 12:00:00   
53702       88587   178148 2139-06-01 19:51:00 2139-06-01 12:00:00   
9025         9230   145065 2148-04-24 12:00:00 2148-04-24 02:35:00   
40393       46228   

In [1606]:
# Add patient age and gender to DF
T = pd.merge(admissions[['SUBJECT_ID','HADM_ID','ADMITTIME']],
             patients[['SUBJECT_ID','DOB','GENDER']],
             on='SUBJECT_ID')
T_old    = T[T.DOB < '2000-01-01']
T        = T[T.DOB > '2000-01-01']

T['AGE'] =((pd.to_datetime(T.ADMITTIME).dt.date - 
            pd.to_datetime(T.DOB).dt.date).values//(3.1689*10**16)).astype(int)
T_old['AGE'] = 90
T = T.append(T_old)
X = X.merge(T[['HADM_ID','AGE','GENDER']],on='HADM_ID')
X.GENDER = X.GENDER.map(lambda x: 0 if x=='M' else 1)

In [1607]:
# add primary dx, secondary dx, tertiary dx, n diagnoses
for i in range(1,4):
    X = X.merge(dx[['HADM_ID','ICD9_CODE']][dx.SEQ_NUM==i],on='HADM_ID')\
         .rename(columns = {'ICD9_CODE':f'DIAGNOSIS_{i}'})
    
X = X.merge(dx.groupby('HADM_ID').SEQ_NUM.max(),on='HADM_ID')\
     .rename(columns = {'SEQ_NUM':f'N_DIAGNOSES'})           \
     .set_index('HADM_ID')

# Fill missing values

In [1608]:
for col in X.columns:
    print(f'{col}:\n\t{X[col].isna().sum()}')

ADMISSION_TYPE:
	0
ADMISSION_LOCATION:
	0
LANGUAGE:
	24334
ETHNICITY:
	0
DIAGNOSIS:
	11
LOS:
	0
DOA:
	0
AGE:
	0
GENDER:
	0
DIAGNOSIS_1:
	0
DIAGNOSIS_2:
	0
DIAGNOSIS_3:
	0
N_DIAGNOSES:
	0


### Fill Missing dx

In [1609]:
X[X.DIAGNOSIS.isna()].DIAGNOSIS_1.unique()

array(['1551', '99673', '1640', '25083', '4280', '5849', '5939', '40391'],
      dtype=object)

In [1610]:
diagnosis_na_dict = {'1551' :'LIVER CANCER' ,
                     '99673':'RENAL OTHER'  ,
                     '1640' :'LUNG CANCER'  ,
                     '25083':'DM'           ,                    
                     '4280' :'HEART FAILURE',
                     '5849' :'RENAL FAILURE',
                     '5939' :'RENAL OTHER'  ,
                     '40391':'HTN'          }

X.DIAGNOSIS = X[['DIAGNOSIS','DIAGNOSIS_1']].set_index('DIAGNOSIS_1').DIAGNOSIS\
                                            .fillna(diagnosis_na_dict).values

### Fill Missing Admisssion Location

In [1611]:
# narrow admission categories
admission_location_map = {'EMERGENCY ROOM ADMIT'     :'ED'      ,
                          'PHYS REFERRAL/NORMAL DELI':'REFERRAL',
                          'CLINIC REFERRAL/PREMATURE':'REFERRAL',
                          'TRANSFER FROM HOSP/EXTRAM':'TRANSFER',
                          'TRANSFER FROM SKILLED NUR':'TRANSFER',
                          '** INFO NOT AVAILABLE **' : np.nan   ,
                          'HMO REFERRAL/SICK '       :'REFERRAL',
                          'TRANSFER FROM OTHER HEALT':'TRANSFER',
                          'TRSF WITHIN THIS FACILITY':'TRANSFER'}
X.ADMISSION_LOCATION = X.ADMISSION_LOCATION.map(admission_location_map)
X.ADMISSION_TYPE     = X.ADMISSION_TYPE.map(lambda x: 'EMERGENCY' if x=='URGENT' else x)

In [1612]:
X[pd.isna(X.ADMISSION_LOCATION)].DIAGNOSIS.value_counts()

NEWBORN                          279
RESPIRATORY FAILURE,UROSEPSIS      1
LIVER LACERATION                   1
PNEUMOPERTONEUM                    1
GUN SHOT WOUND                     1
ANKLE FRACTURE                     1
Name: DIAGNOSIS, dtype: int64

In [1613]:
X[X.DIAGNOSIS=='NEWBORN'].ADMISSION_LOCATION.value_counts()

REFERRAL    7140
TRANSFER       3
Name: ADMISSION_LOCATION, dtype: int64

### Fill Missing Language

In [1614]:
# missing values assumed to be English. Map to binary, indicating presence of language barrier
X.LANGUAGE = X.LANGUAGE.map(lambda x: 1 if x=='ENGL' else 0) 

In [1615]:
admission_na_dict = {'NEWBORN'                      :'REFERRAL',
                     'ANKLE FRACTURE'               :'ED'      ,
                     'LIVER LACERATION'             :'ED'      ,
                     'PNEUMOPERTONEUM'              :'ED'      ,                    
                     'RESPIRATORY FAILURE,UROSEPSIS':'ED'      ,
                     'GUN SHOT WOUND'               :'ED'      }

X.ADMISSION_LOCATION = X[['DIAGNOSIS','ADMISSION_LOCATION']].set_index('DIAGNOSIS').ADMISSION_LOCATION\
                                                            .fillna(admission_na_dict).values

In [1616]:
X.isna().sum().sum()

0

# Narrow categorical
### Broaden ethnicity categories

In [1617]:
# narrow ethnicity categories (english vs. non-english speaking)
ethnicity_map = {'WHITE'                                                   :'WHITE'      , 
                 'UNKNOWN/NOT SPECIFIED'                                   :'UNKNOWN'    , 
                 'MULTI RACE ETHNICITY'                                    :'MULTIRACIAL',
                 'BLACK/AFRICAN AMERICAN'                                  :'BLACK'      , 
                 'HISPANIC OR LATINO'                                      :'LATINX'     ,
                 'PATIENT DECLINED TO ANSWER'                              :'UNKNOWN'    , 
                 'ASIAN'                                                   :'ASIAN'      , 
                 'OTHER'                                                   :'UNKNOWN'     ,
                 'HISPANIC/LATINO - GUATEMALAN'                            :'LATINX'     , 
                 'ASIAN - VIETNAMESE'                                      :'ASIAN'      ,
                 'AMERICAN INDIAN/ALASKA NATIVE'                           :'NATIVE'     , 
                 'WHITE - RUSSIAN'                                         :'WHITE'      ,
                 'HISPANIC/LATINO - PUERTO RICAN'                          :'LATINX'     , 
                 'ASIAN - CHINESE'                                         :'ASIAN'      ,
                 'ASIAN - ASIAN INDIAN'                                    :'ASIAN'      , 
                 'BLACK/AFRICAN'                                           :'BLACK'      ,
                 'HISPANIC/LATINO - SALVADORAN'                            :'LATINX'     , 
                 'HISPANIC/LATINO - DOMINICAN'                             :'LATINX'     ,
                 'UNABLE TO OBTAIN'                                        :'UNKNOWN'    , 
                 'BLACK/CAPE VERDEAN'                                      :'BLACK'      , 
                 'BLACK/HAITIAN'                                           :'BLACK'      ,
                 'WHITE - OTHER EUROPEAN'                                  :'WHITE'      , 
                 'PORTUGUESE'                                              :'WHITE'      ,
                 'SOUTH AMERICAN'                                          :'LATINX'     ,
                 'WHITE - EASTERN EUROPEAN'                                :'WHITE'      , 
                 'CARIBBEAN ISLAND'                                        :'BLACK'      ,
                 'ASIAN - FILIPINO'                                        :'ASIAN'      ,
                 'ASIAN - CAMBODIAN'                                       :'ASIAN'      ,
                 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)'              :'LATINX'     ,
                 'WHITE - BRAZILIAN'                                       :'WHITE'      , 
                 'ASIAN - KOREAN'                                          :'ASIAN'      ,
                 'HISPANIC/LATINO - COLOMBIAN'                             :'LATINX'     , 
                 'ASIAN - JAPANESE'                                        :'ASIAN'      ,
                 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER'               :'NATIVE'     , 
                 'ASIAN - THAI'                                            :'ASIAN'      ,
                 'HISPANIC/LATINO - HONDURAN'                              :'LATINX'     , 
                 'HISPANIC/LATINO - CUBAN'                                 :'LATINX'     ,
                 'MIDDLE EASTERN'                                          :'WHITE'      , 
                 'ASIAN - OTHER'                                           :'ASIAN'      , 
                 'HISPANIC/LATINO - MEXICAN'                               :'LATINX'     ,
                 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE':'NATIVE'     }

X.ETHNICITY = X.ETHNICITY.map(ethnicity_map) #drop unknown after dummies created

### Broaden Dx Categories

In [1618]:
diagnoses = {'STROKE'               :'.*STROKE.*|.*TIA.*|.*CVA.*|.*CEREB.*(VASC|INFARC).*|.*TRANSIENT ISCH.*|'  +
                                     '.*TIA.*'                                                                  ,
             'HEAD_BLEED'           :'.*(BRAIN|CEREB|CRANIAL|ARACHNOID|SUBDURAL|PARIETAL|TEMPORAL|HEAD|'        +
                                     'INTRAPARENCHYMAL|INTRAVENTRICULAR)*.(HEM|BLEED|ANEURYSM).*|'              +
                                     '.*ICB.*|^ANEURYSM.*|^S(A|D)H.*|^ICH($|;)|IVH|.*SUBDURAL.*|.*SAH.*'        ,
             'SEIZURE'              :'.*SE(IZ|ZI)URE.*|.*EPILE(PSY|PTICUS).*'                                   ,
             'AMS'                  :'.*AMS.*|.*MENTAL STATUS.*|.*UNRESPONSIVE.*|ENCEPH.*|DEL.RIUM|ATAXIA|'     +   
                                     '.*MS CHANGE.*|PARAPLEGIA|.*PHASIA|DELTA MS|CONCUSSION|DIPLOPIA|CONFUSION' ,
             'NEURO_DEGENERATIVE'   :'.*MYASTHENIA.*|( |^)ALS( |$)|.*Alzheimer.*|.*PARKINSON.*|.*MULT.*SCLER.*|'+
                                     'G.*BAR.*SYND.*|.*AUTO.*DYS.*'                                             ,
             'NEURO_OTHER'          :'.*MENINGITIS.*|.*EPIDURAL.*|.*CHIARI.*|.*TRIGEM.*|CEREB.*EDEMA|'          +
                                     '.*CSF.*|.*CRANIOTOMY.*|.*PARALYSIS.*|.*HYDROCEPHALUS.*'                   ,
             'PSYCH'                :'.*HANGING.*|.*SUICID.*|.*DEPRESSION.*|.*SCHIZO.*|.*BIPOLAR.*|.*ANXIETY.*|'+
                                     'AGITATION|.*PSYCHOSIS.*|.*ANOREXIA.*'                                     ,
             'CAD'                  :'.*CORONARY ARTERY.*|.*CHEST PAIN.*|.*CORONARY SYNDROME.*|.*ANGINA.*|'     +
                                     '.*CHES*T PAIN.*|.*CAD.*|.*(MAIN|VESSEL).*(DISEASE|OBS).*|.*CATH.*|'       +
                                     '.*CABG.*|.*CARD.*D(Z|X|IS).*|ISCHEMIC REST PAIN|POSITIVE STRESS TEST|CP'  ,   
             'HEART_FAILURE'        :'.*HEART FAILURE.*|.*CARDIOGENIC SHOCK.*|.*CHF.*|.*CARDIOMYOPATHY.*|'      +
                                     '.*VOL.*OVER.*|MYOCARDITIS'                                                ,
             'MI'                   :'.*MYOCARDIAL IN.*|.*STEMI.*|MIMI|IMI|.* MI($| )|SEVERE ISCHEMIA|MI|'      +
                                     '.* MI(\\\\|$| .*||CATH)|.*ST ELEV.*|(.*\\\\|)CATH.*|.*ETT.*MI.*|'         +
                                     'CARDIAC ARRESTYCARDIA.*|MILEFT HEART MI|.*CARD.*ISCH.*'                   ,
             'AORTIC_ANEURYSM'      :'.*(THORA|AORT).*(ANEUR|DIS(S|)ECTION).*|.*AAA.*|DISSECTION|.*BENTAL.*|'   +  
                                      '.*RUP.*AORT.*'                                                           ,
             'VALVE_DX'             :'.*(MITR|AORT|ATR).*(VALVE|STEN|REGU|DEF|INSUF).*|.*ASD.*|.*AVR.*'         ,
             'CARDIAC_ARREST'       :'.*ARREST.*|.*V( ||-)TAC.*|VENTRICULAR (FIB|TACH)|VT|ASYSTOLE'             ,
             'A_FIB'                :'.*ATRIAL (FIB|FLUTTER).*|.*A( ||-)FIB.*|AF'                               ,
             'HYPOTENSION'          :'.*HYPOTENS.*'                                                             ,                      
             'HTN'                  :'.*HYPERTENSI(VE|ON).*|.*HTN.*'                                            ,
             'PERICARDIAL EFFUSION' :'.*PERICARDIAL EFFUSION.*|.*TAMPONADE.*'                                   ,
             'HEART BLOCK'          :'.*(HEART|AV) BLOCK.*'                                                     ,
             'TACHYCARDIA'          :'.*TACHYCARDIA.*|.*SVT.*'                                                  ,
             'DYSRHYTHMIA OTHER'    :'.*AR.*THMIA.*|EKG CHANGES|.*DYSRHYTHMIA.*|.*ICD.*|.*BRADYC.*|PALPITATIONS',
             'ENDOCARDITIS'         :'.*ENDOCARDITIS.*'                                                         ,
             'VASCULAR OTHER'       :'.*(AORT|ARTER|C(A|O)ROTID|ICA).*(STEN|OCC|DIS).*|.*SVC.*|^STENOS.*|'      +
                                     '.*AORTO.*ILIAC.*D.*|VASCULITIS'                                           , 
             'RESPIRATORY_INFECTION':'.*P(N|M)EUMONIA.*|.*PNEMONIA.*|.*PNA.*|.*ASPIR.*|.*BRONCHITIS.*|'         +
                                    'INFLUENZA|FLU'                                                             ,
             'PE'                  :'.*PE.*|.*PULM.*EMBO.*'                                                     ,
             'RESTRICTIVE_LUNG_DX' :'.*ASTHMA.*|.*COPD.*|.*OBST(RUCTIVE|) PULM.*|.*EMPHYSEMA.*|PULM.*FIBROSIS|' +
                                     'BRONCHOSPASM|REACTIVE AIR.*'                                              ,
             'DYSPNEA'             :'.*SHORT.*BREATH.*|.*HYPOX.*|.*DYS(|P)NEA.*|SOB|.*STRIDOR.*|.*TACHYPNEA.*'  ,
             'RESPIRATORY_FAILURE' :'.*RESP.*(FAILURE|ARREST|DISTRESS|INSUF).*|.*ARDS.*|.*ETT.*|.*LUNG.*D(IS|X)',
             'PULMONARY_EDEMA'     :'.*PULMONARY EDEMA.*|.*LUNG INFIL.*'                                        ,
             'HEMO/PNEUMOTHORAX'   :'.*(HYDRO|HEMO|PNEUMO)THORAX.*|.*EMPYEMA.*|.*PLEUR.*EFFUS.*'                ,
             'RESP_OTHER'          :'.*(TRACH|SUBGL|AIR).*(STEN|MALA|FIST).*|.*RESP OTHER.*|.*BRONCH.*TASIS.*|' +
                                    'AIRWAY EDEMA|THROAT SWELLING|TBM|PULM.*SPEC.*|.*BRONCHOPLEURAL.*|'         +
                                    '.*EPIGLOT.*|.*APNEA.*|VOCAL CORD LESION|BLEEDING.*TRACH.*|.*TRACH.*STOMY.*',
             'GIB'                 :'.*(GASTROINTESTINAL|INTRAPERITONEAL|(G|B)I|VARICEAL|RECTAL)( |)B.*|'       +
                                    '.*HEMATEMESIS.*|.*HEMATOCHEZIA.*|.*MELENA.*|.*VARIC.*|BRBPR'               ,
             'GI_OBSTRUCTION'      :'.*(BOWEL|GASTRIC) BLOCK.*|.*OBSTRUCTION.*|.*VOLVULUS.*|.*ILEUS.*'          ,
             'GI_INFLAMMATION'     :'.*(EPIGL.*|CHOL|APEND|PANC|COL|DIVERT|GASTRO).*ITIS.*|DISCITIS'            ,
             'PANCREATIC_DX'       :'.*PANCR.*(CYST|TRANS|FAIL|NEC).*'                                          ,
             'IBD'                 :'.*CROHN.*|.*ULCER.*COLIT.*|UC|IBD'                                         ,
             'GI_INFECTION'        :'.*ABDOMINAL INF.*'                                                         ,
             'GI_OTHER'            :'.*NAUSEA.*|.*VOMIT.*|.*DIARRHEA.*|.*(BOWEL|MESENT.*) ISCHEM.*|'            +
                                    '.*BILE LEAK.*|.*GASTROPARESIS.*|.*SUBGLOTTIC STENOSIS.*|.*N/V.*|'          +
                                    '.*ENTEROCUTANEOUS FISTULA.*|.*RECTAL PROLAPSE.*|.*(BILE|ESOPH).*STRICT.*|' +
                                    'ACUTE ABDOMEN|ESOPHAGEAL RUPTURE|BAR.*ESOPH|ABDOMINAL FISTULA|ACHALASIA|'  +
                                    'SHORT GUT SYNDROME|MESINTERIC ISCHEMIA|ESOPHAGEAL TEAR|ABD.*DISTENSION|'   +
                                    'BIL.*(STRICT|LEAK).*|(DUO|STOM|GAST).*ULCER|COLON POLYP|'                  +
                                    '.*ISCHEM.*(BOWEL|MESENT.*).*|.*PANCREATIC PSEUDOCYST.*|'                   +
                                    '(.* |.*IL|.*COL|^)OSTOMY.*'                                                ,
             'LIVER_FALURE'         :'.*(LIVER|HEPA).*(FAIL|DIS|AMS|TRANS|TX|DX).*|.*HEP.*ENCEPH.*|.*CIRRHOS.*|'+
                                     '.*ASCIT.*|.*JAUNDICE.*|.*HEPATO( |)RENAL.*|.*ELEV.*(LIVER|FT).*'          ,
             'LIVER_OTHER'          :'.*HEP.*(CYST|ITIS).*|.*LIVER LACERATION.*|.*PORTAL VEIN.*|.*OP LIVER.*'   ,
             'RENAL_FAILURE'        :'.*(RENAL|KID).*(FAILURE|DIS|TRANS|TX).*|.*CRF.*|.*HEPATO( |)RENAL.*|'     +
                                     'AV FIST.*|.*ESRD.*|.*UREMIA.*'                                            ,
             'UTI'                  :'.*UTI.*|.*URINARY TRACT INF.*|.*PYELONEPHRITIS.*'                         ,
             'RENAL_OTHER'          :'.*HEMATURIA.*|.*U.*RET.*|CLOTTED AV GRAFT|.*OP.*KIDNEY.*|BLADDER RUPTURE|'+
                                     'RENAL LACERATION|.*HYDRONEPH.*|RENAL OTHER'                               ,
             'SEPSIS'               :'.*SEP(S|T)I.*|.*BACT(E|)REMIA.*|.*NECROTIZING.*|.*POSIT.*BLOOD.*CULT.*'   ,
             'ANEMIA'               :'.*ANEMIA.*'                                                               ,
             'INFECTION_OTHER'      :'.*FEVER.*|.*LEUKOCYTOSIS.*|.*INFECT.*|.*CELLULITIS.*|.*AB(S|)CESS.*|'     +
                                     '.*OSTEOMYELITIS.*|INFECTED.*(GRAFT|FOOT)|BABESIOSIS|MALARIA|.*C.*DIF.*|'  +
                                     '.*ELEV.*(WBC|WHITE).*'                                                    ,
             'ALLERGIC_REACTION'    :'.*ANAPHY.*|.*ALLERG.*|.*ANGI.*DEM.*|.*RASH.*|(FAC|T.*NG).*SWELL.*|'       +
                                     'MAST CELL DEGR.*|.*REACTION.*'                                             ,
             'SKIN_WOUND'           :'NON HEALING ULCER|.*(FOOT|DEC).*ULCER.*|^ULCER$'                          ,
             'CLOT'                 :'.*DVT.*|.*(DEEP|LE|AORT).*THROM.*|.*CLAUDICATION.*|.*DIC.*'               ,
             'BLEEDING_DX'          :'.*DIC.*|.*(ELEV|HIGH).*INR.*|COAG.*PATHY'                                 ,
             'SPLENE_DX'            :'.*SPLE(E|)N(E|IC|).*'                                                     ,
             'SICKLE_CELL'          :'.*SICKLE CELL.*'                                                          ,
             'AUTOIMMUNE'           : '.*LUPUS.*|FAILING GRAFT|MASTOCYTOSIS'                                    ,
             'DKA'                  :'.*(DIABETIC|).*KETO.*|.*DKA.*'                                            ,
             'DM'                   :'.*DIABETES.*|( |^)DM( |$)'                                                ,
             'ELECTROLYTE_DX'       :'.*(HYPER|HYPO)(KALEMIA|NATREMIA|CALCEMIA|GLYCEMIA).*|.*ELECTROLYTE.*|'    +
                                     '.*ACIDO.*|.*ALKI.*'                                                       ,
             'OBESITY'              :'.*OBES.*Y.*'                                                              ,
             'CONSTITUTION'         :'.*DEHYDRATION.*|.*WE(E|A)KNESS.*|.*FAILURE TO THRIVE.*|.*FTT.*|'          +
                                     '.*UNABLE TO AMBULATE.*|.*HYPOTHERMIA.*|.*RHABDO.*|.*CONSTIT.*|'           +
                                     '.*MALNUTRITION.*|.*W.*LOSS.*|.*LETHARGY.*'                                ,
             'ENDOCRINE_OTHER'      :'.*GOITER.*|.*METABOLIC NON.*|.*THYROID STORM|.*ADREN.*'                   ,
             'TRAUMA'               :'.*BIKE.*|.*MOTOR.*VEHICLE.*|.*MV(A|C).*|.*TRAUMA.*|.*TARUMA.*|.*FALL.*|'  +
                                     '.*PEDESTRAIN.*|.*STAB.*|.* CAR ($|.*)|.*INJURY.*|.*ASSAULT.*|.*LACER.*|'  +
                                     'FOUND DOWN|WOUND DEHISCENCE|.*BICYCLE.*|.*NECK.*LAC.*|.*FLAIL.*CHEST.*|'  +
                                     'MULTIPLE FRACTURES|GSW|.*GUN.*SHOT.*|.*MOTORCYCLE.*|.*BITE.*|.*BOAT.*'    ,
             'OD'                   :'.* OD($| .*)|.*OVERDOSE.*|.*TOX.*|.*COCAINE.*|.*NEUROLEPTIC.*|'           +
                                     '.*STEV.*JO.*SYN.*|SJS|.*TRICYCLIC.*|.*ETOH.*'                             ,
             'FRACTURE'             :'.*(ANKLE|HIP|LIMB|ARM|LEG|FEMUR|TIB|FIB|HUMERUS|RADIUS|ACETAB|'           +
                                     'ULNA|RIB|STERNAL).*(FRACTURE|FX).*'                                       ,
             'HEAD_TRAUMA'          :'.*(SKULL|FAC|BRAIN|TEMPO|MANDIB|CEREB|COMP).*(FRAC|FX|LESION|CONTUS).*'   ,
             'WITHDRAWAL'           :'.*WITHDRAW(A|E|)L.*|.*DEL.*TREM.*|( |^)DT( |$)|.*TREMENS.*'               ,
             'JOINT DX'             :'.*OST.*ARTH.*|(.* |^)OA(/| )|.*JOINT DX.*|.* OA$'                         ,
             'ORTHO'                :'.*FIXATION.*|.*(HIP|KNEE).*REPLACEMENT.*|.*ORTHO.*'                       ,
             'POOR_PERFUSION'       :'.*GANGRENE.*|.*(ISCHEM|NECR).*(F(OO|EE)T|LEG|TOE).*|.*LIMB PERF.*|'       +
                                     '(F(OO|EE)T|LEG).*ISCHEM.*|.*D.*FOOT.*|COMP.*SYNDR.*|.*AMPUTATION.*|'      +
                                     '.*PULSELESS.*LEG.*|.*LEG.*NECROSIS.*|.*ART.*INSUF.*'                      ,
             'SPINE_DX'             :'.*(L|C|S|T).*([1-9]|[1][1-9]).*(FX|FRACT|STEN|DEG|LES|SUB).*|.*SC.*LIO.*|'+
                                     '.*(CERV|LUMBAR|THOR|SACRAL|SPIN.*).*(FX|FRACT|STEN|SPOND|LES|SUB|DEG).*|' +
                                     '.*KYPHO.*|.*SPONDYL.*|.*MYELOPATHY.*|.*CORD.*(COMP|SYN).*|.*ITIS.*SPINE'  ,
             'GI_CANCER'            :'.*(ESOPH|COLON|RECT|CHOLANG|GASTR|AMPULLARY|DUOD|STOMA|ABD|SIGMOID)'      +
                                     '.*(CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA).*|.*ADENOMA.*|'               +
                                     '.*(CA|MASS).*(BOWEL|STOMACH|ESOPH|COLON).*'                               ,
             'METASTATIC_CANCER'    :'.*METASTA.*|.* METS.*'                                                    ,
             'NEURO_CANCER'         :'.*(HEAD|BR(AI|IA)N|CRANIAL|CEREB.*|SPIN|OCC|FRONT).*'                     +
                                     '(CA|NEO|MASS|CANCER|TUMOR|CARCINOMA).*|.*MENINGIOMA.*|.*GLIOBLASTOMA.*'   ,
             'RESPIRATORY_CANCER'   :'.*(LUNG|LOBE|PULM|BRONCH|PANCOAST|AIRWAY|LARYN|TRACH).*'                  +
                                     '(CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA|NODULE).*'                       ,
             'PANCREATIC_CANCER'    :'.*PANCREATIC.*(CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA).*|.*GASTRINOMA.*' ,
             'HEME_CANCER'          :'.*LEUKEMIA.*|.*LYMPHOMA.*|.*MYELOMA.*|AML|BLAST CRISIS|.*MYELOFIB.*'      ,
             'BLADDER/RENAL_CANCER' :'.*(BLADDER|RENAL).*(CA|NEO|MASS|CANCER|TUMOR|CARCINOMA).*'                ,
             'LIVER_CANCER'         :'.*(LIVER|HEPAT.*).*(CA|NEO|MASS|CANCER|TUMOR|CARCINOMA|LES).*|HEPATOMA'   ,
             'BREAST_CANCER'        :'.*BREAST (CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA).*'                     ,
             'MALE_REPRO_CANCER'    :'.*(TEST|PROST).*(CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA).*'              ,
             'FEMALE_REPRO_CANCER'  :'.*(UTER|OVAR|ENDOMET|CERVI|ADENEX).*(CA|NEOP|MASS|CANCER|TUMOR|CARC).*'   ,
             'OTHER_CANCER'         :'.*(NECK|MEDIAS|TONG|THOR).*(CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA).*|'  +
                                     '.*CHEMO.*|.*DYSPLASIA.*|SARCOMA'                                          ,
             'SKIN_CANCER'          :'.*SQUAM.*CA.*|.*MELANOM.*'                                                ,
             'ENDOCRINE_CANCER'     :'.*(THYR|PITU).*(CA|NEOPLASM|MASS|CANCER|TUMOR|CARCINOMA|ADENOMA).*|'      +
                                     '.*THY(M|)OM.*'                                                            ,
             'PAIN'                 :'.*(HIP|BACK|HEAD|ABD|LEG|NECK|Q|INTRACT|FLANK|SHOULDER|EPIGASTRIC|ELBOW)' +
                                     '.*(ACHE|PAIN).*|PAIN.*'                                                   ,
             'HERNIA'               :'.*HERNIA.*'                                                               ,
             'MINOR'                :'.*EPISTAXIS.*|.*MINIMALLY INVASIVE.*|.*DYSPHAGIA.*|.*HEMATOMA.*|.*GERD.*|'+
                                     '.*REFLUX.*|.*CONSTIPATION.*|COLD LEG'                                     ,
             'LITHIASES'            :'.*LITHIASIS.*|.*STONE.*|.*COLIC.*|.*BIL.*OBS.*'                           ,
             'FOREIGN_BODY'         :'.*(FB|FOREI).*(ESOPH|THRO|BOD).*|.*(ESOPH|TRACH|AIR).*(FOREI.*|OBS).*|'   +
                                     '.*CHOKING.*|.*IMPACTION.*|.*INGESTION.*'                                  , 
             'ORGAN_DONOR'          :'.*DONOR.*'                                                                ,
             'NON-TRAUMA_ACCIDENTS' :'.*DROWNING.*|.*ELECTROCUTION.*'                                           ,
             'NONSPECIFIC'          :'SHOCK|DIZZINESS|ANASARCA|.*HEM.*P.*SIS.*|NECK SWELLING|.*SYNCOP.*|'       +
                                     'EDEMA|HEMORRHAGE|BLEED(|ING)|COUGH|SORE THROAT|ISCHEMIA|ANEURSYM'         ,
             'PROCEDURE'            :'.*SCOP(Y|IC).*|.*GRAPHY.*|.*ERCP.*|.*LAP.*TOMY.*|.*PROCEDURE.*|'          +
                                     '.*EXPLORATION.*'                                                          ,
             'FEMALE_REPRO_DX'      :'.*FIBROID.*|.*ECTOPIC.*|.*ENDOMETRITIS.*|.*ADNEXAL.*'                     ,
             'PREMATURE'            :'(.*|^)PRE.*MATUR.*'                                                       ,
             'TRANSPLANT'           :'.*TRANSPLANT.*'                                                           ,
             'PREGNANCY_NORMAL'     :'.*NORMAL PREGNANCY.*'                                                     ,
             'NEONATE'              :'.*NEWBORN.*'                                                              ,
             'OTHER'                :'.*CRANIAL DEFECT.*|.*XCISION.*STERN.*'}

In [1619]:
X.DIAGNOSIS = X.DIAGNOSIS.map(lambda x: x[:-4] if x[-4:]=='/SDA' else x, na_action='ignore')

for dx,regx in diagnoses.items():
    X[dx] = X.DIAGNOSIS.str.match(regx).fillna(0).astype(int,errors='ignore')

In [1620]:
# Remaining dx to be mapped
X[X[diagnoses.keys()].sum(axis=1)==0].DIAGNOSIS.value_counts()[:50]

CA                             2
DENS FRACTURE                  2
RULE OUT TUBERCULOSIS          2
VENTRICULAR TACCHYCARDIA       2
SCROTAL EDEMA                  2
PANCREATIC FISTULA             2
NON-HEALING ULCER              2
HYPOVOLEMIA                    2
C SECTION,PRIMARY              2
ESOPHAGEAL DIVERTICULUM        2
HARDWARE FAILURE               2
LARYNGEAL EDEMA                2
SMALL BOWEL TUMOR              2
RIGHT LEG SARCOMA              2
SYPHILIS                       2
CNS MASS                       2
LOWER EXTREMITY EDEMA          2
HEMORRHAGIC SHOCK              2
KNEE PAIN                      2
DEFIB FIRING                   2
MALLORY-WEISS TEAR             2
U/A                            2
RT FRONTAL CONTUSION           2
MEGACOLON                      2
MESOTHELIOMA                   2
LEFT HIP AVASCULAR NECROSIS    2
MULTIFOCAL MOTOR NEUROPATHY    2
FAILURE TO WEAN                2
R/O MEDIASTINITIS              2
MENINGOENCEPHALITIS            2
BRAIN EDEM

### Pull Keywords from DRGs

In [1621]:
# drg mapping
drg_map = {np.nan:np.nan,
         'gastrointestinal obstruction with complications, comorbidities':'GI_SEVERE',
         'percutaneous cardiovascular procedures':'CARDIAC_SX',
         'laparoscopic cholecystectomy w/o c.d.e. w cc':'GI_SX',
         'aicd generator procedures':'CARDIAC_SX',
         'pleural effusion with complications, comorbidities':'RESPIRATORY_SEVERE',
         'hip & femur procedures for trauma except joint replacement':'ORTHO_SX',
         'intervertebral disc excision & decompression':'ORTHO_SX',
         'local excision & removal int fix devices exc hip & femur w cc':'ORTHO_SX',
         'acute leukemia w/o major o.r. procedure w/o cc/mcc':'HEME_CANCER',
         'spinal procedures':'ORTHO_SX',
         'other o.r. procedures for multiple significant trauma w cc':'TRAUMA_SX_SEVERE',
         'craniotomy for trauma age >17':'TRAUMA_SX_SEVERE',
         'diabetes w cc':'DM',
         'stomach, esophageal & duodenal proc age >17 w cc w/o major gi dx':'GI_SX',
         'dorsal & lumbar fusion proc except for curvature of back':'ORTHO_SX',
         'lower extremity & humerus procedures except hip, foot, femur age>17 complications, comorbidities':'ORTHO_SX_SEVERE',
         'fever':'OTHER_ID',
         'cardiac defib implant w cardiac cath w ami/hf/shock w/o mcc':'MI_FAILURE_SX_SEVERE',
         'skin grafts & wound debrid for endoc, nutrit & metab dis w mcc':'SKIN_SOFT_TISSUE_SX',
         'nonspecific cerebrovascular and precerebral occlusion without infarction':'STROKE',
         'sinus & mastoid procedures age >17':'ENT_SX',
         'extensive operating room procedure unrelated to principal diagnosis':'OTHER_SX_SEVERE',
         'perm cardiac pacemaker implant w/o ami, heart failure or shock':'CARDIAC_SX',
         'signs & symptoms of musculoskeletal system & conn tissue w/o mcc':'CONNECTIVE_TISSUE',
         'ear, nose, mouth & throat malignancy w/o cc/mcc':'ENT_CANCER',
         'other infectious & parasitic diseases diagnoses w mcc':'OTHER_ID',
         'septic arthritis w cc':'SEPSIS_ID_SEVERE',
         'perc cardiovasc proc w drug-eluting stent w mcc or 4+ vessels/stents':'CARDIAC_SX',
         'neonate, bwt 1250-1499g w resp dist synd/oth maj resp or maj anom':'RESPIRATORY_NEONATE_SEVERE',
         'respiratory system diagnosis with ventilator support 96+ hrs':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'endocrine disorders w cc':'ENDOCRINE',
         'signs, symptoms & other factors influencing health status':'OTHER',
         'skin, subcutaneous tissue & breast plastic procedures':'BREAST_SX',
         'extreme immaturity or respiratory distress syndrome, neonate':'RESPIRATORY_NEONATE_SEVERE',
         'tendonitis, myositis & bursitis w/o mcc':'CONNECTIVE_TISSUE',
         'vaginal delivery with operating room procedure except sterilization &/or dilation & curettage':'OB_SX',
         'chest pain':'CARDIAC',
         'mouth procedures w cc/mcc':'ENT_SX',
         'hip & femur procedures except major joint w cc':'ORTHO_SX',
         'peripheral vascular disorders w mcc':'VASCULAR',
         'local excision and removal of internal fixation devices except hip and femur without cc':'ORTHO_SX',
         'traumatic stupor & coma, coma <1 hr w mcc':'TRAUMA_SEVERE_COMA',
         'breast procedure for non-malignancy except biopsy & local excision':'BREAST_SX',
         'other vascular procedures':'VASCULAR_SX',
         'carotid artery stent procedure w/o cc/mcc':'CARDIAC_SX',
         'spinal procedures w cc or spinal neurostimulators':'NEURO_SX',
         'other resp system o.r. procedures w cc':'RESPIRATORY_SX',
         'peripheral, vascular disorders with complications, comorbidities':'VASCULAR_SEVERE',
         'other digestive system o.r. procedures w mcc':'GI_SX',
         'hand procedures for injuries':'TRAUMA_SX',
         'dental & oral diseases & injuries':'ENT',
         'pneumothorax w mcc':'RESPIRATORY',
         'seizure age >17 without complications & comorbidities':'NEURO',
         'peripheral vascular disorders w cc':'VASCULAR',
         'peritoneal adhesiolysis w mcc':'GI_SX',
         'cranial/facial procedures w/o cc/mcc':'ENT_SX',
         'skin graft exc for skin ulcer or cellulitis w cc':'SKIN_SOFT_TISSUE_SX',
         'allogeneic bone marrow transplant':'HEME_CANCER_TRANSPLANT_SX',
         'alcohol/drug abuse or dependence, left against medical advice':'PSYCH',
         'epiglottitis':'GI',
         'major small & large bowel procedures w cc':'GI_SX',
         'otitis media & uri w mcc':'ENT_ID',
         'laparoscopic cholecystectomy w/o c.d.e. w/o cc/mcc':'GI_SX',
         'cardiac valve & other major cardiothoracic procedures with cardiac catheter':'CARDIAC_SX_SEVERE',
         'permanent cardiac pacemaker implant w/ ami, heart failure or shock':'MI_FAILURE_SX_SEVERE',
         'traumatic stupor & coma, coma <1 hr age 0-17':'TRAUMA_SEVERE_COMA',
         'nephritis & nephrosis':'GU',
         'other o.r. procedures for multiple significant trauma w mcc':'TRAUMA_SX_SEVERE',
         'fracture, sprain, strain, & dislocation of forearm, hand, foot age >17 with complications, comorbidities':'TRAUMA_SEVERE',
         'hypertension w mcc':'HTN',
         'other disorders of the eye age >17 with complications, comorbidities':'ENT_SEVERE',
         'other digestive system diagnoses w/o cc/mcc':'GI',
         'septicemia w mechanical ventilator w/0 96+ hours age >17':'SEPSIS_ID_VENTILATED_SEVERE',
         'cardiac pacemaker revision except device replacement':'CARDIAC_SX',
         'circulatory disorders with acute myocardial infarction without major complication, discharged alive':'MI_FAILURE_SEVERE',
         'other vascular procedures without complications, cormorbidities':'VASCULAR_SX',
         'cardiac defibrillator implant with cardiac catheterization without acute myocardial infarction (ami), heart failure or shock':'CARDIAC_SX',
         'gastrointestinal vascular insufficiency':'GI',
         'interstitial lung disease':'RESPIRATORY',
         'major head & neck procedures':'ENT_SX_SEVERE',
         'cellulitis w/o mcc':'SKIN_SOFT_TISSUE_ID',
         'transient ischemic attack & precerebral occlusions':'NEURO',
         'cardiac defibrillator implant with cardiac catheterization with acute myocardial infarction (ami), heart failure or shock':'MI_SX_SEVERE',
         'lymphoma & non-acute leukemia w other o.r. proc w mcc':'HEME_SX',
         'penis procedures':'GU_SX',
         'other disorders of the eye w/o mcc':'ENT',
         'acute major eye infections':'ENT_ID_SEVERE',
         'splenectomy w mcc':'HEME_SX',
         'vaginal delivery w/ complicating procedures exc sterilization &/or d&c':'OB_SEVERE',
         'wound debridements for injuries w mcc':'SKIN_SOFT_TISSUE_SX',
         'pancreas, liver & shunt procedures w/o cc/mcc':'LIVER_SX',
         'appendectomy without complicated principal diagnosis with complications, comorbidities':'GI_SX',
         'spinal procedures w/o cc/mcc':'ORTHO_SX',
         'local excision & removal of internal fixation devices except hip & femur':'ORTHO_SX',
         'vein ligation & stripping':'VASCULAR_SX',
         'arthroscopy':'CONNECTIVE_TISSUE_SX',
         'hepatobiliary diagnostic procedures w cc':'LIVER_SX',
         'cocaine abuse & dependence':'PSYCH',
         'heart failure & shock':'CARDIAC_FAILURE_SEVERE',
         'craniotomy & endovascular intracranial procedures w/o cc/mcc':'NEURO_SX',
         'opioid abuse & dependence':'PSYCH',
         'seizures w mcc':'NEURO',
         'amputation for circ sys disorders exc upper limb & toe w mcc':'VASCULAR_SX_SEVERE',
         'wnd debrid & skn grft exc hand, for musculo-conn tiss dis w cc':'CONNECTIVE_TISSUE_SX',
         'acute myocardial infarction, discharged alive w mcc':'MI_FAILURE_SEVERE',
         'other operating room procedures for injuries with complications, comorbidities':'TRAUMA_SX_SEVERE',
         'other female reproductive system operating room procedures':'GU_SX',
         'combined anterior/posterior spinal fusion w cc':'ORTHO_SX',
         'poisoning & toxic effects of drugs age >17 without complications, comorbidities':'OD',
         'major small & large bowel procedures w cc w major gi dx':'GI_SX_SEVERE',
         'peripheral & cranial nerve & other nerve system procedures with complications, comorbidities':'NEURO_SX_SEVERE',
         'transurethral procedures w/o cc/mcc':'GU_SX',
         'myeloprolif disord or poorly diff neopl w maj o.r. proc w mcc':'OTHER_CANCER',
         'tracheostomy w long term mechanical ventilation w/o extensive procedure':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'anal procedures':'GI_SX',
         'cardiac congenital & valvular disorders age >17 without complications, comorbidities':'CARDIAC',
         'partial thickness burns w or w/o skin graft':'SKIN_SOFT_TISSUE',
         'endocrine disorders without complications, comorbidities':'ENDOCRINE',
         'esophagitis, gastroenteritis, & miscellaneous digestive disorders age 0-17':'GI',
         'other circulatory system diagnoses w mcc':'VASCULAR',
         'infectious & parasitic diseases w or procedure':'OTHER_ID',
         'other skin, subcutaneous tissue & breast procedures without complications, comorbidities':'BREAST_SX',
         'pancreas, liver & shunt procedures with complications, comorbidities':'LIVER_SX_SEVERE',
         'sinus & mastoid procedures':'ENT_SX',
         'cholecystectomy except laparoscopic':'GI_SX',
         'fractures of hip & pelvis w mcc':'ORTHO',
         'traumatic injury age >17 with complications, comorbidities':'TRAUMA_SEVERE',
         'hepatic coma & other major acute liver disorders':'LIVER_COMA',
         'pleural effusion w mcc':'RESPIRATORY',
         'urethral stricture':'GU',
         'alcohol abuse & dependence':'PSYCH',
         'o.r. procedures for obesity w cc':'SKIN_SOFT_TISSUE_SX',
         'other musculoskelet sys & conn tiss o.r. proc w cc':'CONNECTIVE_TISSUE_SX',
         'kidney & urinary tract procedures for nonmalignancy':'GU_SX',
         'pelvic evisceration, radical hysterectectomy & radical gyn procs':'GU_SX',
         'malignant breast disorders w mcc':'OTHER_CANCER',
         'bacterial & tuberculous infections of nervous system w/o cc/mcc':'NEURO_ID',
         'other multiple significant trauma w mcc':'TRAUMA_SEVERE',
         'vertigo & other labyrinth disorders':'NEURO',
         'major pancreas, liver & shunt procedures':'LIVER_SX',
         'major hematol/immun diag exc sickle cell crisis & coagul w cc':'HEME_SEVERE',
         'intracranial vascular procedures w pdx hemorrhage w cc':'STROKE_SX_SEVERE',
         'cardiac congenital & valvular disorders age 0-17':'CARDIAC',
         'liver transplant w/o mcc':'LIVER_FAILURE_TRANSPLANT_SEVERE',
         'traumatic injury w/o mcc':'TRAUMA',
         'other disorders of the liver':'LIVER',
         'anal & stomal procedures without complications, comorbidities':'GI_SX',
         'tracheostomy with mechanical ventilation 96+ hours or principal diagnosis except face, mouth, and neck diagnoses':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'simple pneumonia & pleurisy w/o cc/mcc':'RESPIRATORY_ID',
         'minor small & large bowel procedures with complications, comorbidities':'GI_SX',
         'perc cardiovasc proc w non-drug-eluting stent w mcc or 4+ ves/stents':'CARDIAC_SX',
         'o.r. procedures for obesity w mcc':'SKIN_SOFT_TISSUE_SX',
         'admit for renal dialysis':'RENAL_FAILURE',
         'cardiac pacemaker device replacement':'CARDIAC_SX',
         'tonsil & adenoid procedures, except tonsillectomy &/or adenoidectomy only, age >17':'ENT_SX',
         'craniotomy for multiple significant trauma':'TRAUMA_SX_SEVERE',
         'lymphoma and leukemia with major o.r. procedure with cc':'HEME_CANCER_SX_SEVERE',
         'acute & subacute endocarditis w cc':'CARDIAC',
         'extracranial procedures w/o cc/mcc':'ENT_SX',
         'inguinal & femoral hernia procedures age >17 with complications, comorbidities':'OTHER_SX',
         'nonspecific cerebrovascular disorders without complications, comorbidities':'NEURO',
         'hip & femur procedures except major joint w mcc':'ORTHO',
         'nonextensive procedure unrelated to principal diagnosis':'OTHER_SX',
         'dorsal & lumbar fusion proc for curvature of back':'ORTHO_SX',
         'pancreas, liver & shunt procedures without complications, comorbidities':'LIVER_SX',
         'urinary stones & acquired upper urinary tract obstruction':'GU',
         'full term neonate w major problems':'OTHER_NEONATE_SEVERE',
         'fractures of femur':'TRAUMA',
         'diabetes age 0-35':'DM',
         'liver transplant w mcc or intestinal transplant':'LIVER_FAILURE_TRANSPLANT_SEVERE',
         'diverticulitis & diverticulosis':'GI',
         'connective tissue disorders':'CONNECTIVE_TISSUE',
         'other mental health disorders':'PSYCH',
         'ear, nose, mouth & throat malignancy':'ENT_CANCER',
         'lower extrem & humer proc except hip,foot,femur w cc':'ORTHO_SX',
         'wound debridements for injuries':'SKIN_SOFT_TISSUE_SX',
         'major bladder procedures w cc':'GU_SX_SEVERE',
         'syncope & collapse without complications, comorbidities':'OTHER',
         'neonate, birthwt 1500-1999g w major anomaly':'OTHER_NEONATE_SEVERE',
         'other kidney & urinary tract diagnoses w mcc':'GU',
         'acute myocardial infarction, discharged alive w/o cc/mcc':'MI_FAILURE_SEVERE',
         'seizure & headache age >17 without complications, comorbidities':'NEURO',
         'normal newborn':'OTHER_NEONATE',
         'seizure':'NEURO',
         'major hematologic/immunologic diag exc sickle cell crisis & coagul':'HEME_SEVERE',
         'chronic obstructive pulmonary disease w mcc':'RESPIRATORY',
         'atherosclerosis w mcc':'VASCULAR',
         'uterine & adnexa proc for ovarian or adnexal malignancy w mcc':'GU_CANCER_SX',
         'hiv w major hiv related condition':'HIV_ID_SEVERE',
         'other endocrine, nutrition & metabolic operating room procedures without complications, comorbidities':'ENDOCRINE_SX',
         'postoperative or post-traumatic infections w o.r. proc w mcc':'TRAUMA_ID_SX',
         'cholecystectomy except by laparoscope w/o c.d.e. w mcc':'GI_SX',
         'renal failure w mcc':'RENAL_FAILURE_SEVERE',
         'cardiac valve & oth maj cardiothoracic proc w/o card cath w mcc':'CARDIAC_SX',
         'neonate, birthwt 500-749g w/o major procedure':'OTHER_NEONATE',
         'misc disorders of nutrition,metabolism,fluids/electrolytes w/o mcc':'ELECTROLYTE_DX',
         'alcohol/drug abuse or dependence, left ama':'PSYCH',
         'hiv w one signif hiv cond or w/o signif related cond':'HIV_ID_SEVERE',
         'percutaneous cardiovascular procedures with acute myocardial infarction':'MI_FAILURE_SX',
         'allergic reactions w/o mcc':'OTHER',
         'viral meningitis w/o cc/mcc':'NEURO_ID',
         'd&c, conization, laparoscopy & tubal interruption w/o cc/mcc':'GI_SX',
         'kidney & urinary tract infections age >17 with complications, comorbidities':'GU_ID_SEVERE',
         'circulatory disorders except ami, w card cath w/o mcc':'VASCULAR',
         'o.r. proc w diagnoses of other contact w health services w/o cc/mcc':'OTHER_SX',
         'procedure w diag of rehab, aftercare or oth contact w health service':'OTHER_SX',
         'cardiac pacemaker revision except device replacement w/o cc/mcc':'CARDIAC_SX',
         'other skin, subcutaneous tissue & breast procedures with complications, comorbidities':'BREAST_SX_SEVERE',
         'other musculoskeletal system & connective tissue operating room procedures without complications, comorbidities':'CONNECTIVE_TISSUE_SX',
         'knee procedures w pdx of infection with complications and comorbidities':'ORTHO_ID_SEVERE',
         'vagina, cervix & vulva procedures':'GU_SX',
         'medical back problems w/o mcc':'ORTHO',
         'heart assist system implant':'CARDIAC_SX',
         'neonate, birthwt 1000-1249g w/ resp dist synd/oth maj resp or maj anom':'OTHER_NEONATE_SEVERE',
         'septicemia or severe sepsis w/o mv 96+ hours w mcc':'SEPSIS_ID_FAILURE_VENTILATED_SEVERE',
         'acute myocardial infarction, discharged alive w cc':'MI_FAILURE_SEVERE',
         'atherosclerosis w/o mcc':'VASCULAR',
         'extraocular procedures except orbit age >17':'ENT_SX',
         'nervous system infection except viral meningitis':'NEURO_SX',
         'heart failure & shock w cc':'CARDIAC_FAILURE_SEVERE',
         'acute ischemic stroke with use of thrombolytic agent':'STROKE_SEVERE',
         'other o.r. proc of the blood & blood forming organs w mcc':'HEME_SX',
         'rhinoplasty':'ENT_SX',
         'nervous system malignancy':'NEURO_CANCER',
         'atherosclerosis with complications, comorbidities':'VASCULAR_SEVERE',
         'pulmonary embolism':'RESPIRATORY',
         'major chest trauma with complications, comorbidities':'TRAUMA_SEVERE',
         'skin graft exc for skin ulcer or cellulitis w mcc':'SKIN_SOFT_TISSUE_ID_SX',
         'craniotomy age >17 with complications, comorbidities':'NEURO_SX_SEVERE',
         'g.i. hemorrhage w/o cc/mcc':'GI_SEVERE',
         'o.r. proc w diagnoses of other contact w health services w mcc':'OTHER_SX',
         'connective tissue disorders with complications, comorbidities':'CONNECTIVE_TISSUE_SEVERE',
         'digestive malignancy with complications, comorbidities':'GI_CANCER_SEVERE',
         'ventricular shunt procedures':'NEURO_SX',
         'other o.r. procedures for lymphatic/hematopietic/other neoplasms':'HEME_CANCER_SX',
         'peritoneal adhesiolysis':'GI_SX',
         'prostatic o.r. procedure unrelated to principal diagnosis w mcc':'OTHER_SX',
         'pancreas transplant':'GI_TRANSPLANT_SX',
         'coronary bypass with cardiac catheterization without mcv diagnosis':'CARDIAC_SX_SEVERE',
         'cholecystectomy except by laparoscope without common duct exploration with complications, comorbidities':'GI_SX_SEVERE',
         'acute anxiety & delirium states':'PSYCH',
         'cardiac arrest, unexplained w cc':'MI_FAILURE_SEVERE',
         'other kidney & urinary tract procedures w/o cc/mcc':'GU_SX',
         'fx, sprn, strn & disl except femur, hip, pelvis & thigh w mcc':'ORTHO_SX',
         'non-bacterial infect of nervous sys exc viral meningitis w cc':'NEURO_ID',
         'fractures of hip & pelvis w/o mcc':'TRAUMA',
         'thyroid, parathyroid & thyroglossal procedures w/o cc/mcc':'ENDOCRINE_SX',
         'craniotomy age >17 except for trauma':'NEURO_SX',
         'other nervous system & related procedures':'NEURO_SX',
         'chronic obstructive pulmonary disease':'RESPIRATORY',
         'cardiac defib implant w cardiac cath w/o ami/hf/shock w/o mcc':'CARDIAC_SX',
         'bilateral or multiple major joint procs of lower extremity w mcc':'ORTHO_SX_SEVERE',
         'disorders of liver except malignancy, cirrhosis, alcoholic hepatitis without complications, comorbidities':'LIVER',
         'chemotherapy w/o acute leukemia as secondary diagnosis w mcc':'HEME_CANCER',
         'tendinitis, myositis & bursitis':'CONNECTIVE_TISSUE',
         'infectious & parasitic diseases including hiv w o.r. procedure':'OTHER_ID',
         'complicated peptic ulcer w/o cc/mcc':'GI_SEVERE',
         'other digestive system diagnoses age >17 without complications, comorbidities':'GI',
         'other digestive system diagnoses w cc':'GI',
         'uterine,adnexa proc for non-ovarian/adnexal malig w cc':'GU_CANCER',
         'other injury, poisoning & toxic effect diagnoses with complications, comorbidities':'OD_SEVERE',
         'peritoneal adhesiolysis w cc':'GI_SX',
         'trauma to the skin, subcut tiss & breast w mcc':'TRAUMA',
         'other musculoskeletal sys & connective tissue diagnoses w mcc':'CONNECTIVE_TISSUE',
         'acute ischemic stroke w use of thrombolytic agent w cc':'STROKE_SEVERE',
         'chronic obstructive pulmonary disease w/o cc/mcc':'RESPIRATORY',
         'lymphoma & non-acute leukemia w mcc':'HEME_CANCER',
         'lymphoma & leukemia with major operating room procedure':'HEME_SX_SEVERE',
         'laparoscopic cholecystectomy without common duct exploration without complications, comorbidities':'GI_SX',
         'alcohol/drug abuse or dependence w/o rehabilitation therapy w/o mcc':'PSYCH',
         'acute ischemic stroke w use of thrombolytic agent w/o cc/mcc':'STROKE_SEVERE',
         'miscellaneous ear, nose, mouth & throat procedures':'ENT',
         'extracranial vascular procedures without cc':'VASCULAR_SX',
         'spinal fusions except cervical with curvature of the spine or malignancy':'ORTHO_SX',
         'major respiratory & chest procedures':'RESPIRATORY_SX_SEVERE',
         'respiratory infections & inflammations w mcc':'RESPIRATORY_ID',
         'rectal resection with complications, comorbidities':'GI_SX_SEVERE',
         'coronary bypass with cardiac catheter':'CARDIAC_SX_SEVERE',
         'tracheostomy w long term mechanical ventilation w extensive procedure':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'shoulder,elbow or forearm proc,exc major joint proc w/o cc/mcc':'ORTHO_SX',
         'skin grafts for injuries w cc/mcc':'SKIN_SOFT_TISSUE_SX',
         'major hematol/immun diag exc sickle cell crisis & coagul w/o cc/mcc':'HEME_SEVERE',
         'bronchiolitis & rsv pneumonia':'RESPIRATORY_ID',
         'postpartum & post abortion diagnoses with operating room procedure':'OB_SX',
         'spinal fus exc cerv w spinal curv/malig/infec or 9+ fus w/o cc/mcc':'ORTHO_SX',
         'cranial & peripheral nerve disorders w mcc':'NEURO',
         'malfunction, reaction & complication of g.i. device or procedure':'GI',
         'other ear, nose, mouth & throat operating room procedures':'ENT_SX',
         'perc cardiovasc proc w/o coronary artery stent w mcc':'CARDIAC_SX',
         'fracture, sprain, strain & dislocation of upper arm, lower leg, except foot age >17 without complications, comorbidities':'TRAUMA',
         'major small & large bowel procedures with complications, comorbidities':'GI_SX_SEVERE',
         'concussion w cc':'NEURO',
         'hepatobiliary diagnostic procedure for malignancy':'LIVER_CANCER_SX',
         'signs & symptoms w mcc':'OTHER',
         'other o.r. proc of the blood & blood forming organs w cc':'HEME_SX',
         'liver transplant':'LIVER_FAILURE_TRANSPLANT_SEVERE',
         'coronary bypass without cardiac catheterization with mcv diagnosis':'CARDIAC_SX_SEVERE',
         'asthma':'RESPIRATORY',
         'interstitial lung disease with complications, comorbidities':'RESPIRATORY_SEVERE',
         'other myeloprolif dis or poorly diff neopl diag w mcc':'OTHER_CANCER',
         'inborn errors of metabolism':'ENDOCRINE',
         'respiratory system diagnosis w ventilator support 96+ hours':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'combined anterior/posterior spinal fusion w mcc':'ORTHO_SX',
         'cardiac valve & other major cardiothoracic proc without cardiac catheter':'CARDIAC_SX_SEVERE',
         'poisoning & toxic effects of drugs w mcc':'OD',
         'other multiple significant trauma w cc':'TRAUMA_SEVERE',
         'peptic ulcer & gastritis':'GI',
         'stomach, esophageal & duodenal proc age >17 w cc w major gi dx':'GI_SX',
         'major joint replacement or reattachment of lower extremity':'ORTHO_SX_SEVERE',
         'inflammation of the male reproductive system w mcc':'GU',
         'peripheral, cranial & autonomic nerve disorders':'NEURO',
         'ectopic pregnancy':'OB',
         'minor small & large bowel procedures w/o cc/mcc':'GI_SX',
         'spinal disorders & injuries w/o cc/mcc':'ORTHO',
         'heart failure & shock w mcc':'CARDIAC_FAILURE_SEVERE',
         'tendon, muscle & other soft tissue procedures':'SKIN_SOFT_TISSUE',
         'ventricular shunt procedures w cc':'NEURO_SX',
         'major joint replacement or reattachment of lower extremity w/o mcc':'ORTHO_SX_SEVERE',
         'seizure age >17 w complications & comorbidities':'NEURO_SEVERE',
         'headaches w/o mcc':'OTHER',
         'organic mental health disturbances':'PSYCH',
         'tracheostomy for face,mouth & neck diagnoses w cc':'ENT_SX',
         'pneumothorax with complications, comorbidities':'RESPIRATORY_SEVERE',
         'other respiratory system operating room procedures with complications, comorbidities':'RESPIRATORY_SX_SEVERE',
         'schizophrenia':'PSYCH_SEVERE',
         'intracranial vascular procedures w pdx hemorrhage w mcc':'STROKE_SX_SEVERE',
         'connective tissue disorders w cc':'CONNECTIVE_TISSUE',
         'g.i. hemorrhage w mcc':'GI_SEVERE',
         'malignancy of hepatobiliary system or pancreas':'LIVER_CANCER',
         'other o.r. procedures for injuries w mcc':'TRAUMA_SX',
         'major thoracic & abdominal vascular procedures':'OTHER_SX_SEVERE',
         'biopsies of musculoskeletal system & connective tissue w mcc':'CONNECTIVE_TISSUE_SX',
         'upper limb & toe amputation for circulatory system disorders':'VASCULAR_SX_SEVERE',
         'cardiac valve & oth maj cardiothoracic proc w card cath w/o cc/mcc':'CARDIAC_SX_SEVERE',
         'laparoscopic cholecystectomy w/o c.d.e. w mcc':'GI_SX',
         'intra cranial vascular procedure with principal diagnosis of hemorrhage':'STROKE_SX_SEVERE',
         'headaches w mcc':'OTHER',
         'cranial/facial procedures w cc/mcc':'ENT_SX',
         'acute & subacute endocarditis w mcc':'CARDIAC',
         'chemo w acute leukemia as sdx w cc or high dose chemo agent':'HEME_CANCER',
         'amputation of lower limb except toes':'ORTHO_SX',
         'transurethral prostatectomy':'GU_SX',
         'uterine & adnexa procedures for non-ovarian & non-adnexal malig':'GU_SX',
         'cardiac arrest':'CARDIAC_FAILURE_SEVERE',
         'disorders of the biliary tract w/o cc/mcc':'GI',
         'percutaneous cardiovascular procedure with non drug-eluting stent without mcv diagnosis':'CARDIAC_SX',
         'ventricular shunt procedures w/o cc/mcc':'NEURO_SX',
         'extensive procedure unrelated to principal diagnosis':'OTHER_SX_SEVERE',
         'other hepatobiliary, pancreas & abdominal procedures':'LIVER_SX',
         'cholecystectomy with common duct exploration with complications, comorbidities':'GI_SX_SEVERE',
         'peripheral vascular disorders w/o cc/mcc':'VASCULAR',
         'hip & femur procedures except major joint age >17 without complications, comorbidities':'ORTHO_SX',
         'menstrual & other female reproductive system disorders':'GU',
         'permanent cardiac pacemaker implant w mcc':'CARDIAC_SX',
         'other ear, nose, mouth & throat diagnoses age >17':'ENT',
         'major esophageal disorders w/o cc/mcc':'GI_SEVERE',
         'female reproductive system malignancy':'GU_CANCER',
         'disorders of pancreas except malignancy w/o cc/mcc':'GI_CANCER',
         'disorders of the biliary tract with complications, comorbidities':'GI_CANCER_SEVERE',
         'poisoning & toxic effects of drugs w/o mcc':'OD',
         'bacterial & tuberculous infections of nervous system w mcc':'NEURO_ID',
         'perc cardiovasc proc w/o coronary artery stent w/o mcc':'CARDIAC_SX',
         'non-extensive operating room procedure unrelated to principal diagnosis':'OTHER_SX',
         'permanent cardiac pacemaker implant with mcv diagnosis or aicd lead or generator':'CARDIAC_SX',
         'osteomyelitis, septic arthritis & other musculoskeletal infections':'CONNECTIVE_TISSUE_ID',
         'cervical spinal fusion w mcc':'ORTHO_SX',
         'circumcision age 0-17':'GU_SX',
         'extracranial procedures w cc':'ENT_SX',
         'viral meningitis':'GU_ID',
         'hiv w multiple major hiv related conditions':'HIV_ID_SEVERE',
         'g.i. obstruction w/o cc/mcc':'GI',
         'respiratory malignancy':'RESPIRATORY_CANCER',
         'ventricular shunt procedures w mcc':'NEURO_SX',
         'other digestive system diagnoses':'GI',
         'major cardiovascular procedures without complications, comorbidities':'CARDIAC_SX_SEVERE',
         'splenectomy w cc':'HEME_SX',
         'trauma to the skin, subcutaneous tissue & breast age >17 without complications, comorbidities':'TRAUMA',
         'mastectomy procedures':'BREAST_SX',
         'total mastectomy for malignancy with complications, comorbidities':'BREAST_CANCER_SX_SEVERE',
         'laparoscopic cholecystectomy without common duct exploration with complications, comorbidities':'GI_SX',
         'major joint & limb reattachment procedures of lower extremity':'ORTHO_SX_SEVERE',
         'spinal procedures w mcc':'ORTHO_SX',
         'digestive malignancy w cc':'GI_CANCER',
         'red blood cell disorders age >17':'HEME',
         'transurethral procedures w mcc':'GU_SX',
         'adrenal & pituitary procedures':'ENDOCRINE_SX',
         'kidney, ureter & major bladder procedures for non-neoplasm with complications, comorbidities':'GU_SX_SEVERE',
         'myeloprolif disord or poorly diff neopl w other o.r. proc w/o cc/mcc':'OTHER_SX',
         'other resp system o.r. procedures w mcc':'RESPIRATORY_SX',
         'major male pelvic procedures w cc/mcc':'GU_SX_SEVERE',
         'major shoulder/elbow procedure, or other upper extremity procedures with complications, comorbidities':'ORTHO_SX_SEVERE',
         'urinary stones with complications, comorbidities, &/or extracorporeal shock wave lithotripsy':'GU_SEVERE',
         'neonate w other significant problems':'OTHER_NEONATE_SEVERE',
         'ectopic pregnancy procedure':'GU_SX',
         'tracheostomy except for face, mouth, & neck diagnoses':'ENT_SX',
         'soft tissue procedures w mcc':'OTHER_SX',
         'coagulation & platelet disorders':'HEME',
         'major chest & respiratory trauma':'TRAUMA_SEVERE',
         'tonsill & adenoid procedures':'ENT_SX',
         'cardiac valve & oth maj cardiothoracic proc w/o card cath w/o cc/mcc':'CARDIAC_SX_SEVERE',
         'hernia procedures except inguinal & femoral age >17 with complications, comorbidities':'OTHER_SX',
         'splenectomy age >17':'HEME_SX',
         'total mastectomy for malignancy without complications, comorbidities':'OTHER_CANCER',
         'other male reproductive system diagnoses w/o cc/mcc':'GU',
         'lymphoma & non-acute leukemia without complications, comorbidities':'HEME_CANCER',
         'post-op,post-trauma, other device infections w o.r. procedure':'ORTHO_ID',
         'shoulder,elbow or forearm proc,exc major joint proc w cc':'ORTHO_SX',
         'ecmo or trach w mv 96+ hrs or pdx exc face, mouth & neck w maj o.r.':'CARDIAC_FAILURE_VENTILATED_SEVERE',
         'major bladder procedures w/o cc/mcc':'GU_SX_SEVERE',
         'percutaneous cardiovascular procedure with drug-eluding stent without acute myocardial infarction':'CARDIAC_SX',
         'nonspecific cva & precerebral occlusion w/o infarct w mcc':'STROKE',
         'cleft lip & palate repair':'ENT_SX',
         'foot & toe procedures':'ORTHO_SX',
         'esophagitis, gastroenteritis, & miscellaneous digestive disorders age >17 without complications, comorbidities':'GI',
         'aftercare, musculoskeletal system & connective tissue':'CONNECTIVE_TISSUE',
         'kidney & ureter procedures for non-neoplasm w/o cc/mcc':'GU_SX',
         'cervical spinal fusion with complications and comorbidities':'ORTHO_SX_SEVERE',
         'dilation & curettage for non-obstetric procedures':'GU_SX',
         'bronchitis & asthma w/o cc/mcc':'RESPIRATORY',
         'neonate, birthwt > 2499g w/ major anomaly':'OTHER_NEONATE_SEVERE',
         'major cardiovasc procedures w/o mcc':'CARDIAC_SX_SEVERE',
         'cholecystectomy except by laparoscope without common duct exploration without complications, comorbidities':'GI_SX',
         'peripheral & other vascular disorders':'VASCULAR',
         'kidney transplant':'RENAL_FAILURE_TRANSPLANT_SX_SEVERE',
         'major bladder procedures w mcc':'GU_SX_SEVERE',
         'biliary tact procedures except only total cholecyst with or without common duct exploration with complications, comorbidities':'GI_SX',
         'disorders of liver except malig,cirr,alc hepa w/o cc/mcc':'LIVER',
         'lower extremity & humerus procedures except hip, foot, femur age 0-17':'ORTHO_SX',
         'poisoning & toxic effects of drugs age >17 with complications, comorbidities':'OD_SEVERE',
         'cardiac arrhythmia & conduction disorders w mcc':'CARDIAC',
         'intracranial vascular procedures w pdx hemorrhage w/o cc/mcc':'STROKE_SX_SEVERE',
         'biliary tract procedures except only total cholecyst with or without common duct exploration without complications, comorbidities':'GI_SX',
         'moderately extensive procedure unrelated to principal diagnosis':'OTHER_SX',
         'kidney & urinary tract infections w/o mcc':'GU_ID',
         'nervous system neoplasms without complications, comorbidities':'NEURO_CANCER',
         'concussion age 0-17':'NEURO',
         'knee procedures w/o pdx of infection w cc/mcc':'ORTHO_ID_SX',
         'nervous system neoplasms w mcc':'NEURO_CANCER',
         'major chest trauma w/o cc/mcc':'TRAUMA_SEVERE',
         'spinal fusion except cervical without complications and comorbidities':'ORTHO_SX',
         'cardiac arrest, unexplained w/o cc/mcc':'CARDIAC_FAILURE_SEVERE',
         'transurethral prostatectomy with complications, comorbidities':'GU_SX_SEVERE',
         'infectious & parasitic diseases w o.r. procedure w/o cc/mcc':'OTHER_ID_SX',
         'other male reproductive system o.r. proc exc malignancy w/o cc/mcc':'GU_SX',
         'cellulitis & other bacterial skin infections':'SKIN_SOFT_TISSUE_ID',
         'otitis media & upper respiratory infection age 0-17':'ENT_ID',
         'soft tissue procedures without complications, comorbidities':'OTHER_SX',
         'epistaxis w/o mcc':'ENT',
         'inflammation of the male reproductive system w/o mcc':'GU',
         'cardiac defibrillator implant w/o cardiac cath w mcc':'CARDIAC_SX',
         'chemotherapy without acute leukemia as secondary diagnosis':'OTHER_CANCER',
         'other digestive system diagnoses age 0-17':'GI',
         'pulmonary embolism w mcc':'RESPIRATORY',
         'vaginal delivery with sterilization &/or dilation & curettage':'OB',
         'other injury, poisoning & toxic effect diag w/o mcc':'OD',
         'intestinal obstruction':'GI',
         'postoperative & post-traumatic infections w/o mcc':'TRAUMA_ID',
         'bronchitis & asthma age >17 without complications, comorbidities':'RESPIRATORY',
         'disorders of personality & impulse control':'PSYCH',
         'diabetes w mcc':'DM',
         'other disorders of nervous system w/o cc/mcc':'NEURO',
         'pathological fractures & musculoskeletal & connective tissue malignancy':'CONNECTIVE_TISSUE_CANCER',
         'extensive o.r. procedure unrelated to principal diagnosis w cc':'OTHER_SX_SEVERE',
         'combined anterior/posterior spinal fusion w/o cc/mcc':'ORTHO_SX',
         'reticuloendothelial & immunity disorders w cc':'HEME',
         'kidney & urinary tract neoplasms with complications, comorbidities':'GU_CANCER_SEVERE',
         'permanent cardiac pacemaker implant w cc':'CARDIAC_SX',
         'non-extensive o.r. proc unrelated to principal diagnosis w cc':'OTHER_SX',
         'non-extensive o.r. proc unrelated to principal diagnosis w/o cc/mcc':'OTHER_SX',
         'back & neck procedures except spinal fusion without complications and comorbidities':'ORTHO_SX',
         'coronary bypass without cardiac catheterization without mcv diagnosis':'CARDIAC_SX_SEVERE',
         'inflammation of the male reproductive system':'GU',
         'acute myocardial infarction, expired w mcc':'MI_FAILURE_SEVERE',
         'other respiratory system diagnoses w/o mcc':'RESPIRATORY',
         'spinal fusion with complications and comorbidities':'ORTHO_SX',
         'circulatory disorders except acute myocardial infarction, with cardiac catheter & complex diagnoses':'CARDIAC',
         'organic disturbances & mental retardation':'PSYCH',
         'hernia procedures except inguinal, femoral & umbilical':'OTHER_SX',
         'other respiratory system diagnoses w mcc':'RESPIRATORY',
         'connective tissue disorders w mcc':'CONNECTIVE_TISSUE',
         'drug & alcohol abuse or dependence, left against medical advice':'PSYCH',
         'other circulatory system diagnoses with complications, comorbidities':'CARDIAC_SEVERE',
         'revision of hip or knee replacement':'ORTHO_SX',
         'other injury, poisoning & toxic effect diagnoses without complications, comorbidities':'OD',
         'appendectomy w complicated principal diag w cc':'GI_SX_SEVERE',
         'septicemia or severe sepsis w mv 96+ hours':'SEPSIS_ID_SEVERE',
         'epistaxis':'ENT',
         'wound debridements for injuries w/o cc/mcc':'SKIN_SOFT_TISSUE_SX',
         'craniotomy with major device implant or acute complex cns principal diagnosis':'NEURO_SX_SEVERE',
         'infections, female reproductive system w mcc':'GU_ID',
         'multiple sclerosis & cerebellar ataxia w cc':'NEURO',
         'shoulder, upper arm & forearm procedures':'ORTHO_SX',
         'rectal resection w mcc':'GI_SX',
         'complications of treatment without complications, comorbidities':'OTHER_SEVERE',
         'abortion with dilation & curettage, aspiration curettage or hysterotomy':'OB_SX_SEVERE',
         'soft tissue procedures w cc':'OTHER_SX',
         'back & neck proc exc spinal fusion w/o cc/mcc':'ORTHO_SX',
         'cranio w major dev impl/acute complex cns pdx w mcc or chemo implant':'NEURO_SX_SEVERE',
         'spinal fus exc cerv w spinal curv/malig/infec or 9+ fus w mcc':'ORTHO_SX',
         'foot procedures w cc':'ORTHO_SX',
         'syncope & collapse':'OTHER',
         'nervous system neoplasms w/o mcc':'NEURO_CANCER',
         'appendectomy w/o complicated principal diag w/o cc/mcc':'GI_SX',
         'skin graft for skin ulcer or cellulitis w mcc':'SKIN_SOFT_TISSUE_SX',
         'other kidney & urinary tract procedures w cc':'GU_SX',
         'peritoneal adhesiolysis with complications, comorbidities':'GI_SEVERE',
         'coagulation disorders':'HEME',
         'kidney & urinary tract signs & symptoms w mcc':'GU',
         'postpartum & post abortion diagnoses without operating room procedure':'OB',
         'neonate, bwt 2000-2499g w major anomaly':'OTHER_NEONATE_SEVERE',
         'complications of treatment w/o cc/mcc':'OTHER_SEVERE',
         'cardiac defibrillator implant without cardiac catheterization':'CARDIAC_SX',
         'renal failure w cc':'RENAL_FAILURE_SEVERE',
         'major male pelvic procedures':'GU_SX_SEVERE',
         'full term neonate with major problems':'OTHER_NEONATE_SEVERE',
         'septicemia w mechanical ventilator 96+ hours age >17':'SEPSIS_ID_FAILURE_VENTILATED_SEVERE',
         'major joint & limb reattachment procedures of upper extremity':'ORTHO_SX_SEVERE',
         'hepatobiliary diagnostic procedure for non-malignancy':'LIVER_CANCER',
         'other o.r. procedures for multiple significant trauma w/o cc/mcc':'TRAUMA_SX_SEVERE',
         'kidney, ureter & major bladder procedures for neoplasm':'GU',
         'amputation for musculoskeletal sys & conn tissue dis w mcc':'CONNECTIVE_TISSUE_SX_SEVERE',
         'other kidney & urinary tract diagnoses w cc':'GU',
         'infectious & parasitic diseases w o.r. procedure w mcc':'OTHER_ID_SX',
         'lymphoma & leukemia w major o.r. procedure w mcc':'HEME_SX_SEVERE',
         'bilateral or multiple major joint procs of lower extremity w/o mcc':'ORTHO_SX_SEVERE',
         'amputat of lower limb for endocrine,nutrit,& metabol dis w cc':'ENDOCRINE_SX_SEVERE',
         'prematurity with major problems':'OTHER_NEONATE_SEVERE',
         'major joint & limb reattachment proc of upper extremity w cc/mcc':'ORTHO_SX_SEVERE',
         'degenerative nervous system disorders exc mult sclerosis':'NEURO',
         'lymphoma & non-acute leukemia':'HEME_CANCER',
         'viral illness w mcc':'OTHER_ID',
         'cellulitis age >17 with complications, comorbidities':'SKIN_SOFT_TISSUE_ID_SEVERE',
         'other ear, nose, mouth & throat diagnoses w mcc':'ENT',
         'respiratory system signs, symptoms & other diagnoses':'RESPIRATORY',
         'wnd debrid & skn grft exc hand, for musculo-conn tiss dis w/o cc/mcc':'SKIN_SOFT_TISSUE_SX',
         'disorders of pancreas except malignancy w cc':'GI',
         'respiratory infections & inflammations age >17 with complications, comorbidities':'RESPIRATORY_ID_SEVERE',
         'neonate bwt <1500g w major procedure':'OTHER_NEONATE_SEVERE',
         'otitis media & upper respiratory infection age >17 with complications, comorbidities':'ENT_ID_SEVERE',
         'simultaneous pancreas/kidney transplant':'RENAL_FAILURE_TRANSPLANT_SEVERE',
         'major esophageal disorders w cc':'GI_SEVERE',
         'craniotomy & endovascular intracranial procedures w mcc':'NEURO_SX',
         'chemo w acute leukemia as sdx or w high dose chemo agent w mcc':'OTHER_CANCER',
         'other skin, subcut tiss & breast proc w cc':'BREAST',
         'hiv w extensive o.r. procedure w/o mcc':'HIV_ID_SX_SEVERE',
         'other circulatory system diagnoses without complications, comorbidities':'CARDIAC',
         'heart transplant or implant of heart assist system w mcc':'CARDIAC_FAILURE_TRANSPLANT_SEVERE',
         'other digestive system diagnoses age >17 with complications, comorbidities':'GI_SEVERE',
         'septicemia age 0-17':'SEPSIS_ID_SEVERE',
         'cardiac defibrillator implant w/o cardiac cath w/o mcc':'CARDIAC_SX',
         'bronchitis & asthma w cc/mcc':'RESPIRATORY',
         'other multiple significant trauma':'TRAUMA_SEVERE',
         'gastrointestinal hemorrhage with complications, comorbidities':'GI_SEVERE',
         'other kidney & urinary tract diagnoses w/o cc/mcc':'GU',
         'dental extractions & restorations':'ENT',
         'testes procedures, for malignancy':'GU',
         'extracranial vascular procedures':'VASCULAR_SX',
         'anal & stomal procedures w mcc':'GI_SX',
         'breast procedures except mastectomy':'BREAST_SX',
         'malfunction, reaction & comp of cardiac or vasc device or proc':'CARDIAC_SX',
         'major chest procedures w cc':'OTHER_SX_SEVERE',
         'fracture, sprain, strain & dislocation of upper arm, lower leg except foot age >17 with complications, comorbidities':'TRAUMA_SEVERE',
         'dilation & curettage, conization & radio-implant, for malignancy':'GU_CANCER',
         'biopsies of musculoskeletal system & connective tissue w/o cc/mcc':'CONNECTIVE_TISSUE',
         'circulatory disorders with acute myocardial infarction & major complication, discharged alive':'MI_FAILURE_SEVERE',
         'lymphoma and leukemia with major o.r. procedure without cc':'HEME_CANCER_SX_SEVERE',
         'pelvic evisceration, rad hysterectomy & rad vulvectomy w cc/mcc':'GU_SX_SEVERE',
         'other kidney & urinary tract operating room procedures':'GU_SX',
         'other musculoskelet sys & conn tiss o.r. proc w mcc':'CONNECTIVE_TISSUE_SX',
         'neonate, transferred <5 days old, not born here':'OTHER_NEONATE',
         'other endocrine, nutrit & metab o.r. proc w/o cc/mcc':'ENDOCRINE_SX',
         'heart transplant or implant of heart assist system':'CARDIAC_FAILURE_TRANSPLANT_SX_SEVERE',
         'acute leukemia w/o major o.r. procedure w cc':'HEME_CANCER',
         'respiratory signs & symptoms with complications, comorbidities':'RESPIRATORY_SEVERE',
         'skin grafts & wound debrid for endoc, nutrit & metab dis w cc':'ENDOCRINE_SX',
         'other o.r. procedures for injuries w/o cc/mcc':'TRAUMA_SX',
         'pleural effusion w/o cc/mcc':'RESPIRATORY',
         'cardiac valve & oth maj cardiothoracic proc w card cath w mcc':'CARDIAC_SX_SEVERE',
         'other disorders of nervous system':'NEURO',
         'aftercare, musculoskeletal system & connective tissue w cc':'CONNECTIVE_TISSUE',
         'disorders of pancreas except malignancy':'GI',
         'stomach, esophageal & duodenal proc w mcc':'GI_SX',
         'medical back problems w mcc':'ORTHO',
         'inguinal & femoral hernia procedures w cc':'OTHER_SX',
         'complications of treatment with complications, comorbidities':'OTHER_SEVERE',
         'other bladder procedures':'GU_SX',
         'brain contusion/laceration & compliated skull fx, coma < 1 hr or no coma':'TRAUMA_SEVERE_COMA',
         'hand or wrist proc, except major thumb or joint proc w cc/mcc':'ORTHO_SX',
         'mouth procedures without complications, comorbidities':'ENT_SX',
         'postoperative & post-traumatic infections':'OTHER_ID_SX',
         'upper limb & toe amputation for circ system disorders w cc':'VASCULAR_SX_SEVERE',
         'simple pneumonia & pleurisy age 0-17':'RESPIRATORY_ID',
         'other ear, nose, mouth,throat & cranial/facial diagnoses':'ENT',
         'cardiac pacemaker device replacement w/o mcc':'CARDIAC_SX',
         'rectal resection w/o cc/mcc':'GI_SX',
         'simple pneumonia & pleurisy age >17 without complications, comorbidities':'RESPIRATORY',
         'other vascular procedures without complications and comorbidities':'VASCULAR_SX',
         'cholecystectomy except by laparoscope w/o c.d.e. w cc':'GI_SX',
         'other circulatory system diagnoses w cc':'CARDIAC',
         'neonate, birthwt 2000-2499g w other significant condition':'OTHER_NEONATE_SEVERE',
         'specific cerebrovascular disorders except transient ischemic attack':'NEURO',
         'laparoscopy & incisional tubal interruption':'GU_SX',
         'testes & scrotal procedures':'GU_SX',
         'inflammatory bowel disease':'GI',
         'hiv w multiple significant hiv related conditions':'HIV_ID_SEVERE',
         'major head & neck procedures w/o cc/mcc':'ENT_SX_SEVERE',
         'sialoadenectomy':'ENT_SX',
         'o.r. procedures for obesity w/o cc/mcc':'SKIN_SOFT_TISSUE_SX',
         'skin graft exc for skin ulcer or cellulitis w/o cc/mcc':'SKIN_SOFT_TISSUE_ID_SX',
         'skin debridement w mcc':'SKIN_SOFT_TISSUE_SX',
         'other factors influencing health status':'OTHER',
         'postpartum & post abortion diagnoses w/o o.r. procedure':'OB',
         'major chest procedures':'OTHER_SX_SEVERE',
         'allergic reactions w mcc':'OTHER',
         'other skin, subcut tiss & breast proc w/o cc/mcc':'BREAST_SX',
         'interstitial lung disease w/o cc/mcc':'RESPIRATORY',
         'spinal fusion except cervical w mcc':'ORTHO',
         'traumatic stupor & coma, coma <1 hr w cc':'TRAUMA_SEVERE_COMA',
         'g.i. hemorrhage w cc':'GI_SEVERE',
         'major skin disorders w/o mcc':'SKIN_SOFT_TISSUE_SEVERE',
         'cardiac defib implant w cardiac cath w/o ami/hf/shock w mcc':'CARDIAC_SX',
         'viral illness w/o mcc':'OTHER_ID',
         'spinal fus exc cerv w spinal curv/malig/infec or 9+ fus w cc':'ORTHO_SX_SEVERE',
         'other ear, nose, mouth & throat diagnoses w cc':'ENT',
         'musculoskeletal malignancy & pathol fractures d/t muscskel malig':'CONNECTIVE_TISSUE_CANCER',
         'neonate, birthwt 2000-2499g w congenital/perinatal infection':'OTHER_NEONATE_ID',
         'signs & symptoms w/o mcc':'OTHER',
         'cardiac arrest, unexplained':'CARDIAC_FAILURE_SEVERE',
         'orbital procedures':'ENT',
         'hip & femur procedures except major joint age >17 with complications, comorbidities':'ORTHO_SX',
         'other circulatory system operating room procedures':'CARDIAC_SX',
         'dental & oral dis except extractions & restorations age >17':'ENT_SX',
         'poisoning of medicinal agents':'OD',
         'signs & symptoms of musculoskeletal system & connective tissue':'CONNECTIVE_TISSUE',
         'anal & stomal procedures with complications, comorbidities':'GI_SX_SEVERE',
         'skin graft &/or debridement for skin ulcer or cellulitis with complications, comorbidities':'SKIN_SOFT_TISSUE_ID_SX_SEVERE',
         'biopsies of musculoskeletal system & connective tissue':'CONNECTIVE_TISSUE',
         'acute ischemic stroke w use of thrombolytic agent w mcc':'STROKE_SEVERE',
         'alcohol/drug abuse or dependence with complications and comorbidities':'PSYCH_SEVERE',
         'cirrhosis & alcoholic hepatitis w cc':'LIVER',
         'procedures for obesity':'SKIN_SOFT_TISSUE_SX',
         'anal & stomal procedures w/o cc/mcc':'GI_SX',
         'lymphoma & leukemia w major o.r. procedure w cc':'HEME_CANCER_SX_SEVERE',
         'malnutrition, failure to thrive & other nutritional disorders':'ELECTROLYTE_DX',
         'urinary stones w/o esw lithotripsy w/o mcc':'GU',
         'major bladder procedures':'GU_SX_SEVERE',
         'other kidney & urinary tract diagnoses, signs & symptoms':'GU',
         'ear, nose, mouth & throat malignancy w cc':'ENT_CANCER',
         'cranial & peripheral nerve disorders with complications, comorbidities':'NEURO_SEVERE',
         'other o.r. proc of the blood & blood forming organs w/o cc/mcc':'HEME_SX',
         'hiv with major related condition':'HIV_ID_SEVERE',
         'other ear, nose, mouth & throat procedures':'ENT_SX',
         'diabetes age >35':'DM',
         'alcohol/drug abuse or dependence without rehabilitation therapy without complications and comorbidities':'PSYCH',
         'other heart assist system implant':'CARDIAC_SX_SEVERE',
         'concussion, closed skull fx nos, uncomplicated intracranial injury, coma <1 hr or no com':'TRAUMA',
         'other operating room procedures of the blood and blood forming organs':'HEME_SX',
         'biliary tract proc except only cholecyst w or w/o c.d.e. w mcc':'GI_SX',
         'other kidney & urinary tract procedures w mcc':'GU_SX',
         'respiratory signs & symptoms':'RESPIRATORY',
         'angina pectoris & coronary atherosclerosis':'CARDIAC',
         'other infectious & parasitic diseases diagnoses':'OTHER_ID',
         'other digestive system operating room procedures with complications, comorbidities':'GI_SX_SEVERE',
         'tracheostomy for face,mouth & neck diagnoses w/o cc/mcc':'RESPIRATORY_SX',
         'other endocrine disorders':'ENDOCRINE',
         'viral meningitis w cc/mcc':'NEURO_ID',
         'percutaneous cardiovascular procedures w/o ami':'CARDIAC_SX',
         'connective tissue disorders w/o cc/mcc':'CONNECTIVE_TISSUE',
         'spinal fusion except cervical w/o mcc':'ORTHO_SX',
         'tonsillectomy &/or adenoidectomy only, age >17':'ENT_SX',
         'epistaxis w mcc':'ENT',
         'skin graft &/or debridement except for skin ulcer or cellulitis with complications, comorbidities':'SKIN_SOFT_TISSUE_SX_SEVERE',
         'traumatic stupor & coma, coma >1 hr w/o cc/mcc':'TRAUMA_SEVERE_COMA',
         'cranio w major dev impl/acute complex cns pdx w/o mcc':'NEURO_SX_SEVERE',
         'diabetes w/o cc/mcc':'DM',
         'myeloproliferative disorder or poorly diffused neoplasms with other operating room procedures':'OTHER_CANCER_SX',
         'back & neck proc exc spinal fusion w cc/mcc or disc device/neurostim':'ORTHO_SX',
         'major gastrointestinal disorders & peritoneal infections w mcc':'GI_ID_SEVERE',
         'minor skin disorders without complications, comorbidities':'SKIN_SOFT_TISSUE',
         'percutaneous cardiovascular procedures without acute myocardial infarction, without coronary artery stent implant':'CARDIAC_SX',
         'cardiac pacemaker device replacement w mcc':'CARDIAC_SX',
         'spinal disorders & injuries':'ORTHO',
         'cesarean section w cc/mcc':'OB_SX',
         'minor small & large bowel procedures w cc':'GI_SX',
         'kidney & urinary tract signs & symptoms age >17 with complications, comorbidities':'GU_SEVERE',
         'respiratory neoplasms':'RESPIRATORY_CANCER',
         'appendectomy with complicated principal diagnosis with complications, comorbidities':'ENT_SX_SEVERE',
         'adrenal & pituitary procedures w/o cc/mcc':'ENDOCRINE_SX',
         'percutaneous cardiovascular procedure with mcv diagnosis':'CARDIAC_SX',
         'cardiac arrhythmia & conduction disorders with complications, comorbidities':'CARDIAC_SEVERE',
         'thyroid, parathyroid & thyroglossal procedures':'ENDOCRINE',
         'hiv w major related condition w mcc':'HIV_ID_SEVERE',
         'major gastrointestinal & peritoneal infections':'GI_ID_SEVERE',
         'liver transplant and/or intenstinal transplant':'LIVER_FAILURE_TRANSPLANT_SX_SEVERE',
         'other antepartum diagnoses':'OB',
         'cirrhosis & alcoholic hepatitis w mcc':'LIVER',
         'major esophageal disorders':'GI_SEVERE',
         'other complications of treatment':'OTHER_SEVERE',
         'acute leukemia without major operating room procedure age >17':'HEME_CANCER',
         'minor skin disorders w mcc':'SKIN_SOFT_TISSUE',
         'nervous system neoplasms with complications, comorbidities':'NEURO_CANCER_SEVERE',
         'thyroid, parathyroid & thyroglossal procedures w cc':'ENDOCRINE_SX',
         'osteomyelitis w cc':'CONNECTIVE_TISSUE',
         'inborn and other disorders of metabolism':'ENDOCRINE',
         'inguinal & femoral hernia procedures w mcc':'OTHER_SX',
         'infections of the upper respiratory tract':'RESPIRATORY_ID',
         'other musculoskelet sys & conn tiss o.r. proc w/o cc/mcc':'CONNECTIVE_TISSUE',
         'circulatory disorders except ami, w card cath w mcc':'CARDIAC_SX',
         'coronary bypass w/o cardiac cath w/o mcc':'CARDIAC_SX_SEVERE',
         'traumatic stupor & coma, coma >1 hr w cc':'TRAUMA_SEVERE_COMA',
         'orbital procedures w cc/mcc':'ENT_SX',
         'eye disorders except major infections':'ENT',
         'back & neck procedures except spinal fusion with complications and comorbidities':'ORTHO_SX_SEVERE',
         'other kidney, urinary tract & related procedures':'GU_SX',
         'pathological fractures & musculoskelet & conn tiss malig w cc':'CONNECTIVE_TISSUE',
         'non-bacterial infect of nervous sys exc viral meningitis w mcc':'NEURO_ID',
         'major chest trauma w mcc':'TRAUMA_SEVERE',
         'soft tissue procedures with complications, comorbidities':'SKIN_SOFT_TISSUE_SX_SEVERE',
         'uncomplicated peptic ulcer w mcc':'GI',
         'testes procedures, non-malignancy age >17':'GU_SX',
         'kidney & ureter procedures for non-neoplasm w cc':'GU_SX',
         'other infectious & parasitic diseases diagnoses w cc':'OTHER_ID',
         'uterine & adnexa procedures for ovarian & adnexal malignancy':'GU_ID',
         'skin grafts for injuries w/o cc/mcc':'TRAUMA_SX',
         'reticuloendothelial & immunity disorders without complications, comorbidities':'HEME',
         'bone diseases & arthropathies w mcc':'CONNECTIVE_TISSUE',
         'other musculoskeletel system & connective tissue procedures':'CONNECTIVE_TISSUE',
         'respiratory neoplasms w mcc':'RESPIRATORY_CANCER',
         'percutaneous cardiovascular procedure with non drug-eluting stent with mcv diagnosis':'CARDIAC_SX',
         'other endocrine, nutrit & metab o.r. proc w cc':'ENDOCRINE_SX',
         'other musculoskeletal sys & connective tissue diagnoses w/o cc/mcc':'CONNECTIVE_TISSUE',
         'thyroid procedures':'ENDOCRINE',
         'major skin disorders with complications, comorbidities':'SKIN_SOFT_TISSUE_SEVERE',
         'disorders of liver except malig,cirr,alc hepa w mcc':'LIVER',
         'other circulatory system o.r. procedures':'CARDIAC_SX',
         'tendonitis, myositis & bursitis w mcc':'CONNECTIVE_TISSUE',
         'hypertensive encephalopathy w cc':'HTN_SEVERE',
         'neonate, bwt 1500-1999g w resp dist synd/oth maj resp cond':'RESPIRATORY_NEONATE_SEVERE',
         'circulatory disorders except acute myocardial infarction, with cardiac catheter without complex diagnoses':'CARDIAC_SX',
         'malignant breast disorders w cc':'BREAST_CANCER',
         'major chest procedures w mcc':'OTHER_SX_SEVERE',
         'major joint replacement or reattachment of lower extremity w mcc':'ORTHO_SX_SEVERE',
         'acute leukemia':'HEME',
         'kidney & urinary tract neoplasms w cc':'GU_CANCER',
         'acute leukemia w/o major o.r. procedure w mcc':'HEME_CANCER',
         'revision of hip or knee replacement w mcc':'ORTHO_SX',
         'fever of unknown origin age >17 with complications, comorbidities':'OTHER_ID_SEVERE',
         'traumatic stupor & coma, coma >1 hr':'TRAUMA_SEVERE_COMA',
         'bacterial & tuberculous infections of nervous system w cc':'NEURO_ID',
         'hip joint replacement':'ORTHO_SX',
         'cellulitis age >17 without complications, comorbidities':'SKIN_SOFT_TISSUE_ID',
         'other injury, poisoning & toxic effect diag w mcc':'OD',
         'disorders of the biliary tract w cc':'GI',
         'amputation for musculoskeletal sys & conn tissue dis w cc':'CONNECTIVE_TISSUE_SX_SEVERE',
         'craniotomy for trauma':'TRAUMA_SEVERE',
         'operating room procedure with diagnoses of other contact with health services':'OTHER_SX',
         'full thickness burn without skin graft or inhalation injury with cc or significant trauma':'TRAUMA_SEVERE',
         'stomach, esophageal & duodenal proc w/o cc/mcc':'GI_SX',
         'foot procedures':'ORTHO',
         'cardiomyopathy':'CARDIAC',
         'cardiac arrhythmia & conduction disorders':'CARDIAC',
         'g.i. obstruction w mcc':'GI',
         'multiple significant trauma w/o o.r. procedure':'TRAUMA_SX_SEVERE',
         'd&c, conization, laparoscopy & tubal interruption w cc/mcc':'GU_SX',
         'pneumothorax without complications, comorbidities':'RESPIRATORY',
         'chemotherapy w/o acute leukemia as secondary diagnosis w cc':'OTHER_CANCER',
         'other myeloprolif dis or poorly diff neopl diag w cc':'OTHER_CANCER',
         'disorders of the gallbladder & biliary tract':'GI',
         'other & unspecified gastrointestinal hemorrhage':'GI_SEVERE',
         'other cardiothoracic procedures w cc':'CARDIAC_SX',
         'cellulitis w mcc':'SKIN_SOFT_TISSUE_ID',
         'nonspecific cerebrovascular disorders w/o cc/mcc':'VASCULAR',
         'uterine & adnexa procedures for leiomyoma':'GU_SX',
         'minor bladder procedures with complications, comorbidities':'GU_SX',
         'nontraumatic stupor & coma w/o mcc':'OTHER_COMA',
         'skin grafts for injuries':'SKIN_SOFT_TISSUE_SX',
         'other o.r. procedures for injuries w cc':'TRAUMA_SX',
         'osteomyelitis w mcc':'CONNECTIVE_TISSUE_ID',
         'major male pelvic procedures with complications, comorbidities':'GU_SX_SEVERE',
         'seizure & headache age >17 with complications, comorbidities':'NEURO_SEVERE',
         'nonspecific cerebrovascular disorders w cc':'VASCULAR',
         'degenerative nervous system disorders w mcc':'NEURO',
         'pulmonary edema & respiratory failure':'RESPIRATORY_FAILURE',
         'other hepatobiliary or pancreas o.r. procedures w cc':'LIVER_SX',
         'craniotomy age >17 without complications, comorbidities':'NEURO_SX',
         'neonate, transferred <5 days old, born here':'OTHER_NEONATE',
         'major larynx & trachea procedures':'ENT_SX_SEVERE',
         'respiratory system diagnosis with ventilator support':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'simple pneumonia & pleurisy age >17 with complications, comorbidities':'RESPIRATORY_ID_SEVERE',
         'traumatic stupor & coma, coma <1 hr w/o cc/mcc':'TRAUMA_SEVERE',
         'other vascular procedures w/o cc/mcc':'VASCULAR_SX',
         'myeloprolif disord or poorly diff neopl w maj o.r. proc w/o cc/mcc':'OTHER_CANCER',
         'infections, female reproductive system w/o cc/mcc':'GU_ID',
         'uterine & adnexa procedure for non-malignancy with complications, comorbidities':'GU_SX_SEVERE',
         'pelvic evisceration, radical hysterectomy & radical vulvectomy':'GU_SX_SEVERE',
         'skin graft for skin ulcer or cellulitis w cc':'SKIN_SOFT_TISSUE_ID',
         'uncomplicated peptic ulcer w/o mcc':'GI',
         'other circulatory system procedures':'CARDIAC',
         'fracture of pelvis or dislocation of hip':'TRAUMA',
         'biliary tract proc except only cholecyst w or w/o c.d.e. w/o cc/mcc':'LIVER_SX',
         'respiratory signs & symptoms without complications, comorbidities':'RESPIRATORY',
         'parathyroid procedures':'ENDOCRINE_SX',
         'chemotherapy':'OTHER_CANCER',
         'intracranial hemorrhage':'STROKE_SEVERE',
         'major cardiovasc procedures w mcc':'CARDIAC_SX_SEVERE',
         'stomach, esophageal & duodenal proc w cc':'GI_SX',
         'cardiac pacemaker & defibrillator revision except device replacement':'CARDIAC_SX',
         'aicd lead procedures':'CARDIAC_SX',
         'pneumothorax w/o cc/mcc':'RESPIRATORY',
         'operating room procedure for infectious & parasitic diseases':'OTHER_ID_SX',
         'major chest trauma w cc':'TRAUMA_SEVERE',
         'alcohol/drug abuse or dependence w/o rehabilitation therapy w mcc':'PSYCH',
         'malignancy of hepatobiliary system & pancreas':'LIVER_CANCER',
         'red blood cell disorders w mcc':'HEME',
         'lower extrem & humer proc except hip,foot,femur w mcc':'ORTHO_SX',
         'dental & oral diseases w mcc':'ENT',
         'abdominal pain':'GI',
         'respiratory infections & inflammations w cc':'RESPIRATORY_SX',
         'respiratory neoplasms w cc':'RESPIRATORY_CANCER',
         'hiv w major related condition w/o cc/mcc':'HIV_SEVERE',
         'cardiac arrhythmia & conduction disorders w cc':'CARDIAC',
         'other digestive system diagnoses w mcc':'GI',
         'other vascular procedures w mcc':'VASCULAR',
         'other ear, nose, mouth & throat o.r. procedures w/o cc/mcc':'ENT',
         'fractures of hip & pelvis':'TRAUMA',
         'bilateral or multiple major joint procedures of lower extremity':'ORTHO_SX_SEVERE',
         'major esophageal disorders w mcc':'GI_SEVERE',
         'non-bacterial infections of nervous system exc viral meningitis':'NEURO_ID',
         'circulatory disorders with acute myocardial infarction, expired':'MI_FAILURE_SEVERE',
         'breast biopsy & local excision for non-malignancy':'BREAST',
         'other musculoskeletal sys & connective tissue diagnoses w cc':'CONNECTIVE_TISSUE',
         'ear, nose, mouth & throat malignancy w mcc':'ENT_CANCER',
         'hypertensive encephalopathy w mcc':'HTN_SEVERE',
         'minor skin disorders with complications, comorbidities':'SKIN_SOFT_TISSUE',
         'cholecystectomy except by laparoscope w/o c.d.e. w/o cc/mcc':'GI_SX',
         'other female reproductive system & related procedures':'GU',
         'other skin, subcutaneous tissue & related procedures':'GU',
         'other permanent cardiac pacemaker implant without mcv diagnosis':'CARDIAC',
         'acute & subacute endocarditis':'CARDIAC',
         'hernia procedures except inguinal & femoral w/o cc/mcc':'OTHER_SX',
         'urinary stones w/o esw lithotripsy w mcc':'GU',
         'wound debridements for injuries w cc':'SKIN_SOFT_TISSUE_SX',
         'cranial & peripheral nerve disorders w/o mcc':'NEURO',
         'hypertensive encephalopathy w/o cc/mcc':'HTN_SEVERE',
         'transurethral prostatectomy w/o cc/mcc':'GU_SX',
         'cardiac defib implant w cardiac cath w ami/hf/shock w mcc':'CARDIAC_FAILURE_SX_SEVERE',
         'prematurity w/o major problems':'OTHER_NEONATE_SEVERE',
         'testes procedures w/o cc/mcc':'GU_SX',
         'coronary bypass w cardiac cath w mcc':'CARDIAC_SX_SEVERE',
         'transurethral prostatectomy w cc/mcc':'GU_SX',
         'bpd & oth chronic respiratory dis arising in perinatal period':'RESPIRATORY_NEONATE_SEVERE',
         'other o.r. proc for obstetric diagnoses except delivery diagnoses':'OB_SX',
         'complicated peptic ulcer':'GI_SEVERE',
         'amputation for circulatory system disorders except upper limb & toe':'VASCULAR_SX_SEVERE',
         'other back & neck disorders, fractures & injuries':'TRAUMA',
         'knee & lower leg procedures except foot':'ORTHO',
         'signs & symptoms with complications, comorbidities':'OTHER_SEVERE',
         'trauma to the skin, subcutaneous tissue & breast age 0-17':'TRAUMA',
         'major hematol/immun diag exc sickle cell crisis & coagul w mcc':'HEME_SEVERE',
         'major chest trauma without complications, comorbidities':'TRAUMA_SEVERE',
         'alcoholic liver disease':'LIVER',
         'neonate, birthwt 1500-1999g w or w/o other significant condition':'OTHER_NEONATE',
         'appendectomy w/o complicated principal diag w mcc':'GI_SX',
         'hypovolemia & related electrolyte disorders':'ELECTROLYTE_DX',
         'disequilibrium':'NEURO',
         'neonate, bwt 2000-2499g w resp dist synd/oth maj resp cond':'RESPIRATORY_NEONATE_SEVERE',
         'other mental disorder diagnoses':'PSYCH',
         'major gastrointestinal disorders and peritoneal infections':'GI_SX',
         'lymphoma & leukemia w major o.r. procedure w/o cc/mcc':'HEME_CANCER_SEVERE',
         'kidney & urinary tract neoplasms w mcc':'RENAL_CANCER',
         'cardiac defibrillator implant with cardiac catheterization':'CARDIAC_SX',
         'other respiratory system operating room procedures without complications, comorbidities':'RESPIRATORY_SX',
         'peripheral vascular disorders without complications, comorbidities':'VASCULAR',
         'major small & large bowel procedures w cc w/o major gi dx':'GI_SX_SEVERE',
         'heart failure & shock w/o cc/mcc':'CARDIAC_FAILURE_SEVERE',
         'pulmonary embolism w/o mcc':'RESPIRATORY',
         'traumatic stupor & coma, coma <1 hr age >17 without complications, comorbidities':'TRAUMA_SEVERE',
         'cardiac valve & oth maj cardiothoracic proc w/o card cath w cc':'CARDIAC_SX_SEVERE',
         'wound debridement & skin graft except hand, for musculoskeletal & connective tissue disorders age>17 with complications, comorbidities':'SKIN_SOFT_TISSUE_SX',
         'cardiac valve procedures w/o cardiac catheterization':'CARDIAC_SX',
         'transient ischemia':'CARDIAC',
         'poisoning & toxic effects of drugs age 0-17':'OD',
         'chronic obstructive pulmonary disease w cc':'RESPIRATORY',
         'electrolyte disorders except hypovolemia related':'ELECTROLYTE_DX',
         'skin grafts & wound debridement for endocrine, nutrition & metabolic disorders':'SKIN_SOFT_TISSUE_SX',
         'urethral & transurethral procedures':'GU',
         'respiratory system diagnosis with ventilator support <96 hrs':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'malignancy, female reproductive system w mcc':'GU_CANCER',
         'concussion w mcc':'NEURO',
         'coronary bypass w cardiac cath w/o mcc':'CARDIAC_SX_SEVERE',
         'skin graft &/or debridement except for skin ulcer or cellulitis without complications, comorbidities':'SKIN_SOFT_TISSUE_SX',
         'non-extensive o.r. proc unrelated to principal diagnosis w mcc':'OTHER_SX',
         'other digestive system operating room procedures without complications, comorbidities':'GI_SX',
         'biopsies of musculoskeletal system & connective tissue w cc':'CONNECTIVE_TISSUE',
         'hernia procedures except inguinal & femoral age >17 without complications, comorbidities':'OTHER_SX',
         'neurological eye disorders':'NEURO',
         'other hepatobiliary or pancreas o.r. procedures w mcc':'LIVER_SX',
         'esophagitis, gastroent & misc digest disorders w/o mcc':'GI',
         'other procedures of blood & blood forming organs':'HEME',
         'tracheostomy for face,mouth & neck diagnoses w mcc':'ENT_SX',
         'allergic reactions age >17':'OTHER',
         'cardiac arrhythmia & conduction disorders without complications, comorbidities':'CARDIAC',
         'prematurity without major problems':'OTHER_NEONATE',
         'knee procedures w pdx of infection w mcc':'ORTHO_ID_SX',
         'nonspecific cva & precerebral occlusion w/o infarct w/o mcc':'STROKE',
         'neonate, birthwt 1000-1249g w or w/o other significant condition':'OTHER_NEONATE',
         'coronary bypass without cardiac catheter':'CARDIAC_SX_SEVERE',
         'other myeloproliferative diseases or poorly diffused neoplasm diagnoses with complications, comorbidities':'OTHER_CANCER',
         'nonspecific cerebrovascular disorders with complications, comorbidities':'VASCULAR_SEVERE',
         'traumatic injury w mcc':'TRAUMA',
         'mouth procedures w/o cc/mcc':'ENT_SX',
         'concussion age >17 with complications, comorbidities':'NEURO_SEVERE',
         'intracranial hemorrhage or cerebral infarction w cc':'STROKE_SEVERE',
         'fractures of femur w/o mcc':'TRAUMA',
         'local excision & removal int fix devices of hip & femur w cc/mcc':'ORTHO',
         'sickle cell anemia crisis':'HEME',
         'prostatic operating room procedure unrelated to principal diagnosis':'GU_SX',
         'other male reproductive system & related procedures':'GU_SX',
         'intracranial hemorrhage & stroke w/ infarction':'STROKE_SEVERE',
         'non-bacterial gastroenteritis, nausea & vomiting':'GI_ID',
         'lymphoma & non-acute leukemia w cc':'HEME_CANCER',
         'other disorders of nervous system with complications, comorbidities':'NEURO_SEVERE',
         'coronary bypass w ptca w mcc':'CARDIAC_SEVERE',
         'postoperative or post-traumatic infections w o.r. proc w cc':'TRAUMA_ID',
         'stomach, esophageal, & duodenal procedures age >17 with complications, comorbidities':'GI_SX_SEVERE',
         'other digestive system o.r. procedures w/o cc/mcc':'GI_SX',
         'musculoskeletal & other procedures for multiple significant trauma':'CONNECTIVE_TISSUE_SX_SEVERE',
         'connective tissue disorders without complications, comorbidities':'CONNECTIVE_TISSUE',
         'disorders of the biliary tract without complications, comorbidities':'GI',
         'minor small & large bowel procedures w mcc':'GI_SX',
         'neonate, birthwt 1250-1499g w or w/o other significant condition':'OTHER_NEONATE',
         'other antepartum diagnoses w medical complications':'OB_SEVERE',
         'skin graft for skin & subcutaneous tissue diagnoses':'SKIN_SOFT_TISSUE_SX',
         'postpartum & post abortion diagnoses w o.r. procedure':'OB_SX',
         'nonspecific cva & precerebral occlusion w/o infarct':'STROKE',
         'pancreas, liver & shunt procedures w mcc':'LIVER_SX',
         'peripheral & cranial nerve & other nerve system procedures without complications, comorbidities':'NEURO',
         'malignancy, male reproductive system w mcc':'GU_CANCER',
         'hypertension':'HTN',
         'reticuloendothelial & immunity disorders w/o cc/mcc':'HEME',
         'operating room procedure with principal diagnoses of mental illness':'PSYCH_SX',
         'extensive o.r. procedure unrelated to principal diagnosis w mcc':'OTHER_SX',
         'cardiac valve & oth maj cardiothoracic proc w card cath w cc':'CARDIAC_SX_SEVERE',
         'vagina, cervix & vulva procedures w cc/mcc':'GU_SX',
         'pituitary & adrenal procedures':'ENDOCRINE_SX',
         'other aftercare & convalescence':'OTHER',
         'other skin, subcut tiss & breast proc w mcc':'SKIN_SOFT_TISSUESKIN_SOFT_TISSUE_SX',
         'hip & femur procedures for non-trauma except joint replacement':'ORTHO_SX',
         'benign prostatic hypertrophy with complications, comorbidities':'GU',
         'nutritional & miscellaneous metabolic disorders age >17 with complications, comorbidities':'ELECTROLYTE_DX_SEVERE',
         'coronary bypass w/o cardiac cath w mcc':'CARDIAC_SX_SEVERE',
         'spinal disorders & injuries w cc/mcc':'TRAUMA',
         'other permanent cardiac pacemaker implant or ptca with coronary artery stent implant':'CARDIAC_SX',
         'extraocular procedures except orbit':'ENT_SX',
         'chemotherapy w/ acute leukemia as secondary diagnosis or with use of high-dose chemotherapy agent':'HEME_CANCER',
         'abortion w d&c, aspiration curettage or hysterotomy':'OB_SX',
         'postoperative or post-traumatic infections w o.r. proc w/o cc/mcc':'TRAUMA_ID',
         'uterine & adnexa proc for ovarian or adnexal malignancy w cc':'GU_CANCER_SX',
         'foot procedures w mcc':'ORTHO_SX',
         'hypertensive encephalopathy':'HTN_SEVERE',
         'alcohol/drug abuse or dependence, detoxification or other symptom treatment with complications, comorbidities':'PSYCH_SEVERE',
         'sprains, strains, & dislocations of hip, pelvis & thigh w cc/mcc':'TRAUMA',
         'misc disorders of nutrition,metabolism,fluids/electrolytes w mcc':'ELECTROLYTE_DX',
         'pathological fractures & musculoskelet & conn tiss malig w mcc':'CONNECTIVE_TISSUE',
         'major skin disorders w mcc':'SKIN_SOFT_TISSUE_SEVERE',
         'inflammatory bowel disease w cc':'GI',
         'degenerative nervous system disorders':'NEURO',
         'major hematologic/immunologic dx excep scyle cell crisis & coag':'HEME_SEVERE',
         'vaginal delivery w/ sterilization &/or d&c':'OB',
         'trauma to the skin, subcut tiss & breast w/o mcc':'TRAUMA',
         'pancreas, liver & shunt procedures w cc':'LIVER_SX',
         'acute myocardial infarction, expired w cc':'MI_FAILURE_SEVERE',
         'amputation for musculoskeletal sys & conn tissue dis w/o cc/mcc':'CONNECTIVE_TISSUE_SX_SEVERE',
         'perc cardiovasc proc w non-drug-eluting stent w/o mcc':'CARDIAC_SX',
         'major small & large bowel procedures w mcc':'GI_SX_SEVERE',
         'interstitial lung disease w mcc':'RESPIRATORY',
         'bone diseases & arthropathies w/o mcc':'CONNECTIVE_TISSUE',
         'nontraumatic stupor & coma w mcc':'OTHER_COMA',
         'uterine & adnexa proc for non-malignancy w cc/mcc':'GU_SX',
         'simple pneumonia & pleurisy w cc':'RESPIRATORY_ID',
         'dysequilibrium':'NEURO',
         'other respiratory system diagnoses with complications, comorbidities':'RESPIRATORY_SEVERE',
         'renal failure':'RENAL_FAILURE_SEVERE',
         'lymphoma & non-acute leukemia with complications, comorbidities':'HEME_CANCER_SEVERE',
         'post-operative, post-traumatic, other device infections':'TRAUMA_ID',
         'endocrine disorders w mcc':'ENDOCRINE',
         'fx, sprn, strn & disl except femur, hip, pelvis & thigh w/o mcc':'TRAUMA',
         'major o.r. procedures for lymphatic/hematopietic/other neoplasms':'HEME_CANCER_SX',
         'other cardiac pacemaker implantation':'CARDIAC_SX',
         'coronary bypass with ptca':'CARDIAC_SX_SEVERE',
         'skin ulcers w mcc':'SKIN_SOFT_TISSUE',
         'malignant breast disorders':'BREAST_CANCER',
         'dental & oral diseases w/o cc/mcc':'ENT',
         'complications of treatment w mcc':'OTHER_SEVERE',
         'transurethral procedures w cc':'GU',
         'mastectomy for malignancy w cc/mcc':'BREAST_CANCER_SX',
         'myeloprolif disord or poorly diff neopl w other o.r. proc w cc/mcc':'OTHER_CANCER_SX',
         'hiv with extensive operating room procedure':'HIV_SX_SEVERE',
         'traumatic stupor & coma, coma >1 hr w mcc':'TRAUMA_SEVERE_COMA',
         'osteomyelitis':'CONNECTIVE_TISSUE_ID',
         'lymphoma & non-acute leukemia w/o cc/mcc':'HEME_CANCER',
         'female reproductive system reconstructive procedures':'GU_SX',
         'other cardiothoracic procedures w mcc':'CARDIAC_SX',
         'kidney & urinary tract signs & symptoms w/o mcc':'RENAL',
         'major male pelvic procedures w/o cc/mcc':'GU_SX_SEVERE',
         'multiple sclerosis & cerebellar ataxia':'NEURO',
         'neonatal aftercare':'OTHER_NEONATE',
         'other esophageal disorders':'GI',
         'laparoscopic cholecystectomy':'GI_SX',
         'skin graft, except hand, for musculoskeletal & connective tissue diagnoses':'SKIN_SOFT_TISSUE_SX',
         'other disorders of the eye w mcc':'ENT',
         'septicemia age >17':'SEPSIS_ID_SEVERE',
         'cervical spinal fusion without complications and comorbidities':'ORTHO_SX',
         'renal dialysis access device procedure only':'RENAL',
         'acute myocardial infarction, expired w/o cc/mcc':'MI_FAILURE_SEVERE',
         'neonate, birthwt 1500-1999g w congenital/perinatal infection':'OTHER_NEONATE_ID_SEVERE',
         'traumatic injury age >17 without complications, comorbidities':'TRAUMA',
         'periph/cranial nerve & other nerv syst proc w/o cc/mcc':'NEURO_SX',
         'cardiac valve procedures w/ cardiac catheterization':'CARDIAC_SX',
         'major cardiovascular procedures with complications, comorbidities':'CARDIAC_SX_SEVERE',
         'non-bacterial infections of nervous system except viral meningitis':'NEURO_ID',
         'neonate, birthwt > 2499g w resp dist synd/oth maj resp cond':'RESPIRATORY_NEONATE_SEVERE',
         'toxic effects of non-medicinal substances':'OD',
         'major respiratory infections & inflammations':'RESPIRATORY_ID_SEVERE',
         'vaginal delivery with complicating diagnoses':'OB_SEVERE',
         'percutaneous cardiovascular procedures w/ ami':'MI_FAILURE_SX',
         'major cardiothoracic repair of heart anomaly':'CARDIAC_SX_SEVERE',
         'adrenal & pituitary procedures w cc/mcc':'ENDOCRINE_SX',
         'hypertension w/o mcc':'HTN',
         'lymphoma & non-acute leukemia w other o.r. proc w cc':'HEME_CANCER_SX',
         'cardiac congenital & valvular disorders':'CARDIAC',
         'malignancy, male reproductive system':'GU_CANCER',
         'major biliary tract procedures':'GI_SX_SEVERE',
         'myeloproliferative disorder or poorly diffused neoplasms with major operating room procedures with complications, comorbidities':'OTHER_CANCER_SX',
         'mouth procedures with complications, comorbidities':'ENT_SX_SEVERE',
         'bacterial & tuberculous infections of nervous system':'NEURO_SX',
         'cervical spinal fusion w/o cc/mcc':'ORTHO_SX',
         'disorders of pancreas except malignancy w mcc':'GI',
         'kidney & ureter procedures for neoplasm w mcc':'GU_CANCER_SX',
         'major small & large bowel procedures w/o cc/mcc':'GI_SX_SEVERE',
         'fractures of femur w mcc':'TRAUMA',
         'kidney & ureter procedures for non-neoplasm w mcc':'RENAL_SX',
         'craniotomy with implant of chemo agent or acute complex cns principal diagnosis':'NEURO_CANCER_SX_SEVERE',
         'nonspecific cerebrovascular disorders w mcc':'VASCULAR',
         'extracranial procedures w mcc':'ENT_SX',
         'kidney & urinary tract infections w mcc':'GI_ID',
         'cholecystectomy with common duct exploration without complications, comorbidities':'GI_SX',
         'nutritional & miscellaneous metabolic disorders age >17 without complications, comorbidities':'ELECTROLYTE_DX',
         'spinal fusion except cervical with complications and comorbidities':'ORTHO_SX',
         'infections, female reproductive system w cc':'GU_ID',
         'viral illness':'OTHER_ID',
         'percutaneous cardiovascular procedure with drug-eluding stent with acute myocardial infarction':'MI_FAILURE_SX',
         'o.r. procedure for other complications of treatment':'OTHER_SX_SEVERE',
         'other operating room procedures for injuries without complications, comorbidities':'TRAUMA_SX',
         'intracranial hemorrhage or cerebral infarction w mcc':'STROKE_SEVERE',
         'disorders of the biliary tract w mcc':'GI',
         'respiratory infections & inflammations w/o cc/mcc':'RESPIRATORY_ID',
         'infectious & parasitic diseases w o.r. procedure w cc':'OTHER_ID_SX',
         'other cardiothoracic procedures':'CARDIAC_SX',
         'cardiac congenital & valvular disorders w mcc':'CARDIAC',
         'carotid artery stent procedure':'VASCULAR_SX',
         'other endocrine, nutrit & metab o.r. proc w mcc':'ENDOCRINE',
         'd&c, aspiration curettage or hysterotomy for obstetric diagnoses':'GU_SX_SEVERE',
         'coronary bypass with cardiac catheterization with mcv diagnosis':'CARDIAC_SX_SEVERE',
         'cardiac arrhythmia & conduction disorders w/o cc/mcc':'CARDIAC',
         'migraine & other headaches':'OTHER',
         'radiotherapy':'OTHER_CANCER_SX',
         'esophagitis, gastroenteritis, & miscellaneous digestive disorders age >17 with complications, comorbidities':'GI_SEVERE',
         'cardiac arrest, unexplained w mcc':'CARDIAC_FAILURE',
         'other respiratory diagnosis except signs, symptoms & minor diagnoses':'RESPIRATORY',
         'knee procedures w pdx of infection w cc':'ORTHO',
         'cardiac defibrillator & heart assist anomaly':'CARDIAC_SX',
         'major stomach, esophageal & duodenal procedures':'GI_SEVERE',
         'otitis media & uri w/o mcc':'ENT_ID',
         'kidney & ureter procedures for neoplasm w cc':'GU_CANCER_SX',
         'endocrine disorders w/o cc/mcc':'ENDOCRINE',
         'heart failure':'CARDIAC_FAILURE',
         'ventricular shunt procedures with cc':'NEURO_SX',
         'angina pectoris':'CARDIAC',
         'neonate, birthwt 750-999g w/o major procedure':'OTHER_NEONATE',
         'sprains, strains, & dislocations of hip, pelvis & thigh':'TRAUMA',
         'esophagitis, gastroent & misc digest disorders w mcc':'GI',
         'permanent cardiac pacemaker implant w/o cc/mcc':'CARDIAC_SX',
         'neonate, bwt > 2499g, normal newborn or neonate w other problem':'OTHER_NEONATE_SEVERE',
         'major skin disorders':'SKIN_SOFT_TISSUE_SEVERE',
         'other endocrine, nutrition & metabolic operating room procedures with complications, comorbidities':'ENDOCRINE_SEVERE',
         'malignancy of hepatobiliary system or pancreas w cc':'LIVER_CANCER',
         'limb reattachment, hip and femur procedure for multiple significant trauma':'TRAUMA_SX_SEVERE',
         'contusion, open wound & other trauma to skin & subcutaneous tissue':'TRAUMA',
         'acute myocardial infarction':'MI_FAILURE_SEVERE',
         'minor small & large bowel procedures':'GI_SX',
         'malfunction, reaction & comp of orthopedic device or procedure':'ORTHO',
         'acute adjustment reaction & psychosocial dysfunction':'PSYCH',
         'dilation & curettage, conization except for malignancy':'GU_SX',
         'cardiac congenital & valvular disorders w/o mcc':'CARDIAC_SX',
         'cardiac pacemaker & defibrillator device replacement':'CARDIAC_SX',
         'minor skin disorders w/o mcc':'SKIN_SOFT_TISSUE',
         'intracranial hemorrhage or cerebral infarction w/o cc/mcc':'STROKE_SEVERE',
         'other operating room procedures for multiple significant trauma':'TRAUMA_SX_SEVERE',
         'complications of treatment w cc':'OTHER_SEVERE',
         'aftercare without history of malignancy as secondary diagnosis':'OTHER_CANCER',
         'cardiac catheterization for ischemic heart disease':'CARDIAC_SX',
         'soft tissue procedures w/o cc/mcc':'SKIN_SOFT_TISSUE_SX',
         'percutaneous cardiovascular procedures without acute myocardial infarction, with coronary artery stent implant':'CARDIAC_SX',
         'bipolar disorders':'PSYCH',
         'knee joint replacement':'ORTHO_SX',
         'other respiratory system diagnoses without complications, comorbidities':'RESPIRATORY',
         'cesarean delivery':'OB_SX_SEVERE',
         'other musculoskeletal system & connective tissue diagnosis':'CONNECTIVE_TISSUE',
         'other anemia & disorders of blood & blood forming organs':'HEME',
         'skin ulcers':'SKIN_SOFT_TISSUE',
         'respiratory system diagnosis w ventilator support <96 hours':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'other kidney & urinary tract diagnoses age >17 with complications, comorbidities':'GU_SEVERE',
         'other vascular procedures with complications, cormorbidities':'VASCULAR_SX_SEVERE',
         'amputation for circ sys disorders exc upper limb & toe w cc':'ORTHO_SX_SEVERE',
         'trauma to the skin, subcutaneous tissue & breast age >17 with complications, comorbidities':'TRAUMA_SEVERE',
         'male reproductive system diagnoses except malignancy':'GU',
         'extensive o.r. procedure unrelated to principal diagnosis w/o cc/mcc':'OTHER_SX',
         'other vascular procedures with complications and comorbidities':'VASCULAR_SX_SEVERE',
         'splenectomy':'HEME_SX',
         'kidney & urinary tract procedures for malignancy':'RENAL_CANCER_SX',
         'seizures w/o mcc':'NEURO',
         'female reproductive system infections':'GU_ID',
         'major chest procedures w/o cc/mcc':'OTHER_SX_SEVERE',
         'carotid artery stent procedure w mcc':'VASCULAR',
         'salivary gland procedures':'ENT',
         'respiratory system diagnosis w/ ventilator support 96+ hours':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'lower extremity & humerus procedures except hip, foot, femur age>17 without complications, comorbidities':'ORTHO',
         'inguinal, femoral & umbilical hernia procedures':'OTHER_SX',
         'major cranial/facial bone procedures':'ENT_SEVERE',
         'red blood cell disorders w/o mcc':'HEME',
         'eating disorders':'PSYCH',
         'cardiac pacemaker revision except device replacement w cc':'CARDIAC_SX',
         'hip & femur procedures except major joint w/o cc/mcc':'ORTHO_SX',
         'splenectomy w/o cc/mcc':'HEME_SX',
         'simple pneumonia & pleurisy w mcc':'RESPIRATORY_ID',
         'cardiac catheterization w/ circ disord exc ischemic heart disease':'CARDIAC_SX',
         'periph/cranial nerve & other nerv syst proc w mcc':'NEURO_SX',
         'appendectomy':'GI_SX',
         'other vascular procedures w cc':'VASCULAR_SX',
         'other cardiothoracic procedures w/o cc/mcc':'CARDIAC_SX',
         'complicated peptic ulcer w cc':'GI_SEVERE',
         'kidney & urinary tract malignancy':'GU_CANCER',
         'menstrual & other female reproductive system disorders w cc/mcc':'GU',
         'reticuloendothelial & immunity disorders w mcc':'HEME',
         'uterine,adnexa proc for non-ovarian/adnexal malig w mcc':'GU_SX',
         'autologous bone marrow transplant w cc/mcc':'HEME_TRANSPLANT',
         'facial bone procedures except major cranial/facial bone procedures':'ENT_SX',
         'septicemia or severe sepsis w/o mv 96+ hours w/o mcc':'SEPSIS_ID_SEVERE',
         'hiv w major related condition w cc':'HIV_SEVERE',
         'other ear, nose, mouth & throat o.r. procedures w cc/mcc':'ENT_SX',
         'major thumb or joint procedure, or other hand or wrist procedure with complications, comorbidities':'ORTHO_SX',
         'malignancy, male reproductive system, with complications, comorbidities':'GU_CANCER',
         'coronary bypass w ptca w/o mcc':'CARDIAC_SX',
         'transurethral procedures without complications, comorbidities':'GU_SX',
         'hand & wrist procedures':'ORTHO_SX',
         'cardiac congenital & valvular disorders age >17 with complications, comorbidities':'CARDIAC_SEVERE',
         'cirrhosis & alcoholic hepatitis':'LIVER',
         'other major head & neck procedures':'ENT_SX_SEVERE',
         'uterine & adnexa procedures for non-malignancy except leiomyoma':'GU_SX',
         'interstitial lung disease w cc':'RESPIRATORY',
         'uterine & adnexa procedure for non-malignancy without complications, comorbidities':'GU_SX',
         'local excision and removal of internal fixation devices except hip and femur with cc':'ORTHO_SX',
         'cervical spinal fusion w cc':'ORTHO_SX',
         'other infectious & parasitic diseases diagnoses w/o cc/mcc':'OTHER_ID',
         'minor stomach, esophageal & duodenal procedures':'GI_SX',
         'kidney & urinary tract infections age >17 without complications, comorbidities':'GU_ID',
         'inguinal & femoral hernia procedures w/o cc/mcc':'OTHER_SX',
         'dental & oral diseases w cc':'ENT',
         'neonate bwt <500g or ga <24 weeks':'OTHER_NEONATE_SEVERE',
         'coronary bypass w/ cardiac cath or percutaneous cardiac procedure':'CARDIAC_SX_SEVERE',
         'malignancy of hepatobiliary system or pancreas w/o cc/mcc':'LIVER_CANCER',
         'digestive malignancy w mcc':'GI_CANCER',
         'intracranial hemorrhage or cerebral infarction':'STROKE_SEVERE',
         'inflammatory bowel disease w mcc':'GI',
         'ecmo or tracheostomy with mechanical ventilation 96+ hours or principal diagnoses except face, mouth and neck diagnoses with major operating room procedure':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'pleural effusion w cc':'RESPIRATORY',
         'neonate, birthwt > 2499g w congenital/perinatal infection':'OTHER_NEONATE_ID_SEVERE',
         'amputation for musculoskeletal system & connective tissue disorders':'CONNECTIVE_TISSUE_SX_SEVERE',
         'myeloproliferative disorder or poorly diffused neoplasms with major operating room procedures without complications, comorbidities operating room procedures without complications, comorbidities':'OTHER_CANCER_SX',
         'major gastrointestinal disorders & peritoneal infections w/o cc/mcc':'GI_ID_SEVERE',
         'bronchitis & asthma age >17 with complications, comorbidities':'RESPIRATORY_SEVERE',
         'other ear, nose, mouth & throat diagnoses w/o cc/mcc':'ENT',
         'sinus & mastoid procedures w cc/mcc':'ENT_SX',
         'testes procedures w cc/mcc':'GU_SX',
         'wnd debrid & skn grft exc hand, for musculo-conn tiss dis w mcc':'SKIN_SOFT_TISSUE_SX',
         'multiple sclerosis & cerebellar ataxia w/o cc/mcc':'NEURO',
         'other pneumonia':'RESPIRATORY',
         'viral illness age >17':'OTHER_ID',
         'urethral procedures w cc/mcc':'GU_SX',
         'prematurity w major problems':'OTHER_NEONATE_SEVERE',
         'other respiratory & chest procedures':'RESPIRATORY_SX',
         'other disorders of nervous system w mcc':'NEURO',
         'aftercare, musculoskeletal system & connective tissue w mcc':'CONNECTIVE_TISSUE',
         'major small & large bowel procedures without complications, comorbidities':'GU_SX_SEVERE',
         'shoulder, elbow or forearm procedure, except major joint procedures without complications, comorbidities':'ORTHO_SX',
         'other disorders of the eye age >17 without complications, comorbidities':'ENT',
         'revision of hip or knee replacement w cc':'ORTHO_SX',
         'major small & large bowel procedures':'GI_SX_SEVERE',
         'other kidney & urinary tract diagnoses age 0-17':'GU',
         'craniotomy except for trauma':'TRAUMA_SX',
         'concussion age >17 without complications, comorbidities':'NEURO',
         'postpartum & post abortion diagnoses w/o procedure':'OB',
         'malignancy of hepatobiliary system or pancreas w mcc':'LIVER_CANCER',
         'multiple sclerosis & other demyelinating diseases':'NEURO',
         'lymphatic & other malignancies & neoplasms of uncertain behavior':'HEME_CANCER',
         'traumatic stupor & coma, coma <1 hr age >17 with complications, comorbidities':'TRAUMA_SEVERE',
         'reticuloendothelial & immunity disorders with complications, comorbidities':'HEME_SEVERE',
         'upper limb & toe amputation for circ system disorders w mcc':'VASCULAR_SX_SEVERE',
         'kidney & urinary tract infections':'GU_ID',
         'kidney & ureter procedures for neoplasm w/o cc/mcc':'GU_CANCER_SX',
         'neonates, died or transferred to another acute care facility':'OTHER_NEONATE_SEVERE',
         'pancreas transplants':'GI_TRANSPLANT',
         'spinal procedures without cc':'ORTHO_SX',
         'neonate, bwt 2000-2499g, normal newborn or neonate w other problem':'OTHER_NEONATE',
         'other disorders of nervous system without complication, comorbidities':'NEURO',
         'postoperative or post-traumatic infections w or procedure':'TRAUMA_SX',
         'non-malignant breast disorders w/o cc/mcc':'BREAST',
         'craniotomy age 0-17':'NEURO_SX',
         'other multiple significant trauma w/o cc/mcc':'TRAUMA_SEVERE',
         'postoperative & post-traumatic infections w mcc':'TRAUMA_ID',
         'abortion without dilation & curettage':'OB',
         'signs & symptoms of musculoskeletal system & conn tissue w mcc':'CONNECTIVE_TISSUE',
         'permanent cardiac pacemaker implant with acute myocardial infarction, heart failure or shock, or aicd lead or generator proc':'MI_SX_SEVERE',
         'uterine & adnexa procedures for ovarian or adnexal malignancy':'GU_CANCER_SX',
         'minor bladder procedures w mcc':'GU_SX',
         'lower extrem & humer proc except hip,foot,femur w/o cc/mcc':'ORTHO_SX',
         'digestive malignancy':'GI_CANCER',
         'trach w mv 96+ hrs or pdx exc face, mouth & neck w/o maj o.r.':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'amputation of lower limb for endocrine, nutrition & metabolic disorders':'ENDOCRINE_SX_SEVERE',
         'renal failure w/o cc/mcc':'RENAL_FAILURE',
         'foot procedures w/o cc/mcc':'ORTHO_SX',
         'other procedures for endocrine, nutrititional & metabolic disorders':'ENDOCRINE_SX',
         'shoulder,elbow or forearm proc,exc major joint proc w mcc':'ORTHO_SX',
         'vaginal delivery':'OB',
         'other male reproductive system o.r. proc exc malignancy w cc/mcc':'GU_SX',
         'other myeloproliferative diseases or poorly diffused neoplasm diagnoses without complications, comorbidities':'OTHER_CANCER',
         'other female reproductive system o.r. procedures w cc/mcc':'GU_SX',
         'nasal trauma & deformity':'ENT',
         'bone marrow transplant':'HEME_CANCER_TRANSPLANT',
         'head trauma w/ coma > 1 hr or hemorrhage':'TRAUMA_SEVERE_COMA',
         'tracheostomy for face, mouth & neck diagnoses':'ENT_SX',
         'other disorders of nervous system w cc':'NEURO',
         'non-bacterial infect of nervous sys exc viral meningitis w/o cc/mcc':'NEURO_ID',
         'other drug abuse & dependence':'PSYCH',
         'other digestive system o.r. procedures w cc':'GI_SX',
         'syncope & collapse with complications, comorbidities':'OTHER',
         'carotid artery stent procedure w cc':'CARDIAC_SX',
         'allergic reactions':'OTHER',
         'endocrine disorders with complications, comorbidities':'ENDOCRINE_SEVERE',
         'perc cardiovasc proc w drug-eluting stent w/o mcc':'CARDIAC_SX',
         'other circulatory system diagnoses':'CARDIAC',
         'neonate with other significant problems':'OTHER_NEONATE_SEVERE',
         'breast biopsy, local excision & other breast procedures w cc/mcc':'BREAST_SX',
         'uterine, adnexa procedures for non-ovarian/adnexal malignancy with complications, comorbidities':'GU_CANCER_SX',
         'extracranial vascular procedures with cc':'NEURO_SX',
         'local excision & removal int fix devices exc hip & femur w mcc':'ORTHO_SX',
         'other antepartum diagnoses w/o medical complications':'OB',
         'rectal resection w cc':'GI_SX',
         'extensive abdominal/thoraic procedures for mult significant trauma':'TRAUMA_SX_SEVERE',
         'other skin, subcutaneous tissue & breast disorders':'SKIN_SOFT_TISSUE',
         'other hepatobiliary or pancreas operating room procedures':'LIVER_SX',
         'disorders of liver except malignancy, cirrhosis, alcoholic hepatitis with complications, comorbidities':'LIVER_SEVERE',
         'periph/cranial nerve & other nerv syst proc w cc or periph neurostim':'NEURO_SX',
         'fracture or dislocation except femur, pelvis & back':'TRAUMA',
         'other circulatory system diagnoses w/o cc/mcc':'CARDIAC',
         'medical back problems':'ORTHO',
         'other injury, poisoning & toxic effect diagnoses':'OD',
         'local excision & removal int fix devices exc hip & femur w/o cc/mcc':'ORTHO_SX',
         'tracheostomy with mechanical ventilation 96+ hours or principal diagnosis except face, mouth and neck diagnoses without major operating room procedure':'RESPIRATORY_FAILURE_VENTILATED_SEVERE',
         'lymphoma & non-acute leukemia with other operating room procedures with complications, comorbidities':'HEME_CANCER_SX_SEVERE',
         'neonate, birthwt > 2499g w other significant condition':'OTHER_NEONATE_SEVERE',
         'other musculoskeletal system & connective tissue diagnoses':'CONNECTIVE_TISSUE',
         'major gastrointestinal disorders & peritoneal infections w cc':'GI_ID',
         'otitis media & upper respiratory infection age > 17 without complications, comorbidities':'ENT_ID',
         'hiv w or w/o other related condition':'HIV_ID',
         'multiple sclerosis & cerebellar ataxia w mcc':'NEURO',
         'craniotomy & endovascular intracranial procedures w cc':'NEURO_SX',
         'septicemia & disseminated infections':'SEPSIS_ID_SEVERE',
         'malfunction, reaction & complic of genitourinary device or proc':'GU_SX',
         'cholecystectomy w c.d.e. w mcc':'GI_SX',
         'pneumothorax w cc':'RESPIRATORY',
         'disorders of liver except malig,cirr,alc hepa w cc':'LIVER',
         'digestive malignancy w/o cc/mcc':'GI_CANCER',
         'concussion w/o cc/mcc':'NEURO',
         'hernia procedures except inguinal & femoral w cc':'OTHER_SX',
         'hernia procedures except inguinal & femoral w mcc':'OTHER_SX',
         'amputat of lower limb for endocrine,nutrit,& metabol dis w mcc':'ENDOCRINE_SX_SEVERE',
         'other infectious & parasitic diseases':'OTHER_ID',
         'ear, nose, mouth throat, cranial/facial malignancies':'ENT_CANCER',
         'cranial & peripheral nerve disorders without complications, comorbidities':'NEURO',
         'hepatobiliary diagnostic procedures w mcc':'LIVER_SX',
         'cesarean section with complications, comorbidities':'OB_SX_SEVERE',
         'uterine,adnexa proc for non-ovarian/adnexal malig w/o cc/mcc':'GU_CANCER_SX',
         'stomach, esophageal & duodenal procedures age >17 without complications, comorbidities':'GI_SX',
         'thyroid, parathyroid & thyroglossal procedures w mcc':'ENDOCRINE_SX',
         'other musculoskeletal system & connective tissue operating room procedures with complications, comorbidities':'CONNECTIVE_TISSUE_SX_SEVERE',
         'operating room procedures for obesity':'SKIN_SOFT_TISSUE_SX',
         'degenerative nervous system disorders w/o mcc':'NEURO',
         'deep vein thrombophlebitis w cc/mcc':'VASCULAR',
         'coronary bypass w/o cardiac cath or percutaneous cardiac procedure':'CARDIAC_SX_SEVERE',
         'appendectomy w complicated principal diag w mcc':'GI_SX_SEVERE',
         'vaginal delivery w complicating diagnoses':'OB',
         'cervical spinal fusion & other back/neck proc exc disc excis/decomp':'ORTHO_SX',
         'myeloprolif disord or poorly diff neopl w maj o.r. proc w cc':'OTHER_CANCER',
         'hiv w extensive o.r. procedure w mcc':'HIV_SX_SEVERE',
         'nontraumatic stupor & coma':'OTHER_COMA',
         'cva & precerebral occulusion w/ infarct':'STROKE_SEVERE',
         'g.i. obstruction w cc':'GI',
         'other resp system o.r. procedures w/o cc/mcc':'RESPIRATORY_SX',
         'cardiac pacemaker revision except device replacement w mcc':'CARDIAC_SX',
         'major depressive disorders & other/unspecified psychoses':'PSYCH',
         'transurethral procedures with complications, comorbidities':'GU_SX_SEVERE',
         'non-extensive burns':'SKIN_SOFT_TISSUE',
         'knee procedures without pdx of infection':'ORTHO_SX',
         'diabetes':'DM',
         'biliary tract proc except only cholecyst w or w/o c.d.e. w cc':'GI_SX',
         'limb reattachment, hip & femur proc for multiple significant trauma':'TRAUMA_SX_SEVERE',
         'other antepartum diagnoses with medical complications':'OB',
         'spinal fusion without complications and comorbidities':'ORTHO_SX',
         'spinal procedures with cc':'ORTHO_SX',
         'respiratory infections & inflammations age >17 without complications, comorbidities':'RESPIRATORY_ID',
         'complicated peptic ulcer w mcc':'GI_SEVERE',
         'other disorders of the eye age 0-17':'ENT',
         'peritoneal adhesiolysis w/o cc/mcc':'GI_SX',
         'hiv with or without other related condition':'HIV',
         'o.r. proc w diagnoses of other contact w health services w cc':'OTHER_SX',
         'eye procedures except orbit':'ENT_SX',
         'psychoses':'PSYCH',
         'other digestive system & abdominal procedures':'GI_SX',
         'penis procedures w cc/mcc':'GU_SX',
         'major head & neck procedures w cc/mcc or major device':'ENT_SX_SEVERE',
         'gastrointestinal hemorrhage without complications, comorbidities':'GI_SEVERE'}

In [1622]:
# Map drg and tags
def get_descriptor(s,descriptor):
    return 1 if s[-len(descriptor):]==descriptor else 0
def strip_descriptor(s,descriptor):
    return s[:-len(descriptor)-1] if s[-len(descriptor):]==descriptor else s

tags = ['COMA','SEVERE','VENTILATED','SX','TRANSPLANT','FAILURE','CANCER','ID','NEONATE']

drg_copy = drg.copy()
drg_copy.DESCRIPTION = drg.DESCRIPTION.map(drg_map)
for tag in tags:
    drg_copy['TAG_'+tag] = drg_copy.DESCRIPTION.map(lambda x: get_descriptor(x,tag), na_action='ignore')
    drg_copy.DESCRIPTION = drg_copy.DESCRIPTION.map(lambda x: strip_descriptor(x,tag), na_action='ignore')

tags = ['TAG_'+tag for tag in tags]
drg_copy  = drg_copy[tags+['DESCRIPTION','HADM_ID']].drop_duplicates()
drg_copy  = pd.get_dummies(drg_copy,prefix='TAG',columns=['DESCRIPTION'])\
              .groupby('HADM_ID').sum().applymap(lambda x: 1 if x>=1 else 0)
drg_copy.drop('TAG_OTHER',axis=1,inplace=True)
X = X.merge(drg_copy,on='HADM_ID',how='left')
X[drg_copy.columns] = X[drg_copy.columns].fillna(0)

In [1623]:
# Merge overlapping columns
overlap = ['STROKE','TRANSPLANT','NEONATE','DM','ELECTROLYTE_DX','HTN','MI','OD','PSYCH','TRAUMA','SEPSIS']

for dx in overlap:
    X[dx] = X[[dx,'TAG_'+dx]].apply(lambda x: 1 if sum(x)>0 else 0,axis=1)
    X.drop('TAG_'+dx,axis=1,inplace=True)

In [1624]:
# Number of Rows that Have no Dx Mapped

tags = ['TAG_COMA','TAG_SEVERE','TAG_VENTILATED','TAG_SX','TAG_FAILURE','TAG_CANCER','TAG_ID','TAG_BREAST', 
        'TAG_CARDIAC','TAG_CONNECTIVE_TISSUE','TAG_ENDOCRINE','TAG_ENT','TAG_GI','TAG_GU','TAG_HEME','TAG_HIV',
        'TAG_LIVER','TAG_NEURO','TAG_OB','TAG_ORTHO','TAG_RENAL','TAG_RESPIRATORY','TAG_SKIN_SOFT_TISSUE',
        'TAG_VASCULAR']
dxs = tags + list(diagnoses.keys())
 
X[X[dxs].sum(axis=1)==0] 

,ADMISSION_TYPE,ADMISSION_LOCATION,LANGUAGE,ETHNICITY,DIAGNOSIS,LOS,DOA,AGE,GENDER,DIAGNOSIS_1,...,TAG_HIV,TAG_LIVER,TAG_NEURO,TAG_OB,TAG_ORTHO,TAG_RENAL,TAG_RESPIRATORY,TAG_SKIN_SOFT_TISSUE,TAG_SKIN_SOFT_TISSUESKIN_SOFT_TISSUE,TAG_VASCULAR
HADM_ID,,,,,,,,,,,,,,,,,,,,,


### Broaden Primary, Secondary, and Tertiary Dx

In [1625]:
icd_map = {'^00[1-9]'                   :'GI_INFECTION'                  ,
           '^0(1|2|3([0-7]|8-9)|4[0-1])':'BACTERIAL_INFECTION_OTHER'     ,
           '^038'                       :'SEPSIS'                        ,
           '^04[2-4]'                   :'HIV'                           ,
           '^0(4[5-9]|[5-7])'           :'VIRAL_INFECTION'               ,
           '^08'                        :'ARTHROPOD_BORN_INFECTION'      ,
           '^09'                        :'VENERIAL_DISEASE'              ,
           '^(10|1[2-3])'               :'INFECTION_OTHER'               ,
           '^11'                        :'FUNGAL_INFECTION'              ,
           '^1[4-5].*'                  :'GI_CANCER'                     ,
           '^16.*'                      :'RESPIRATORY_CANCER'            ,
           '^17.*'                      :'CONNECTIVE_TISSUE_CANCER'      ,
           '^18.*'                      :'GU_CANCER'                     ,
           '^(19|23).*'                 :'OTHER_CANCER'                  ,
           '^20.*'                      :'HEME_CANCER'                   ,
           '^2[1-2].*'                  :'BENIGN_NEOPLASM'               ,
           '^24[4-7].*'                 :'THYROID_DX'                    ,
           '^2(49|50).*'                :'DM'                            ,
           '^(2[6-7]|783).*'            :'NUTRIRION_METABOLIC_DX'        ,
           '^28[0-5].*'                 :'ANEMIA'                        ,
           '^28[6-7].*'                 :'BLEEDING'                      ,
           '^28[8-9].*'                 :'HEME_OTHER'                    ,
           '^29[0-4].*'                 :'ORGANIC_PSYCH'                 ,
           '^3(0|1[0:9]).*'             :'PSYCH_CONDITIONS'              ,
           '^33[0-7].*'                 :'NEURO_DEGENERATIVE'            ,
           '^35.*'                      :'PERIPHERAL_NEURO_DX'           ,
           '^(3(39|3[6-8])|781).*'      :'NEURO_OTHER'                   ,
           '^39.*'                      :'CARDIAC_RHEUMATISM'            ,
           '^40.*'                      :'HTN'                           ,
           '^41[1-4].*'                 :'CARDIAC_ISCHEMIA'              ,
           '^(42|785).*'                :'CARDIAC_OTHER'                 ,
           '^43.*'                      :'CEREBROVASCULAR_DX'            ,
           '^44.*'                      :'ARTERIAL_DX'                   ,
           '^45.*'                      :'VENOUS DX'                     ,
           '^4(6|8).*'                  :'RESPIRATORY_INFECTION'         ,
           '^49.*'                      :'RESTRICTIVE_LUNG_DX'           ,
           '^50.*'                      :'PNEUMOCONIOSES'                ,
           '^(47|51([1-7]|9)|786)'      :'RESPIRATORY_OTHER'             ,
           '^518.*'                     :'RESPERATORY_ACUTE'             ,
           '^5[2-3].*'                  :'GI_UPPER_DX'                   ,
           '^54.*'                      :'GI_HERNIA'                     ,
           '^55.*'                      :'GI_INFLAMMATION'               ,
           '^(56|579|787).*'            :'GI_OTHER'                      ,
           '^57[0-3].*'                 :'LIVER_DX'                      ,
           '^57[4-6].*'                 :'GALLBALDDER_DX'                ,
           '^577.*'                     :'PANCREAS_DX'                   ,
           '^(5[8-9]|788).*'            :'RENAL_DX'                      ,
           '^6[0-2].*'                  :'REPRODUCTIVE_DX'               ,
           '^63.*'                      :'PREGNANCY_ABORTIVE'            ,
           '^6(4|[6-7]).*'              :'PREGNANCY_COMPLICATIONS'       ,
           '^63.*'                      :'PREGNANCY_NORMAL'              ,
           '^68.*'                      :'INTEGUMENTARY_INFECTION'       ,
           '^(69|70|782).*'             :'INTEGUMENTARY_OTHER'           ,
           '^(71|72[5-9]).*'            :'ARTHROPATHY'                   ,
           '^72[0-4].*'                 :'DORSOPATHY'                    ,
           '^73.*'                      :'MUSCULOSKELETAL_OTHER'         ,
           '^7[4-5].*'                  :'CONGENITAL_ANOMALY'            ,
           '^7[6-7]'                    :'NEONATE_CONDITIONS'            ,
           '^7(8(0|4||9)|9[6-7])'       :'NONSPECIFIC'                   ,
           '^80[5-9]'                   :'SPINE_FRACTURE'                ,
           '^81'                        :'ARM_FRACTURE'                  ,
           '^82'                        :'LEG_FRACTURE'                  ,
           '^8[3-4]'                    :'SPRAINS_DISLOCATIONS'          ,
           '^(85|80[0-4])'              :'HEAD_TRAUMA'                   ,
           '^86'                        :'TRUNK_INJURY'                  ,
           '^8[6-7]'                    :'OPEN_WOUND'                    ,
           '^9([0-2]|[4-5])'            :'TRAUMA_OTHER'                  ,
           '^93'                        :'FOREIGN_BODY'                  ,
           '^9[6-7]'                    :'TOXICITY_MEDICATION'           ,
           '^98'                        :'TOXICITY_OTHER'                ,
           '^99[0-5]'                   :'EXPOSURE_OTHER'                ,
           '^996[0-5]'                  :'DEVICE_COMPLICATION_MECHANICAL',
           '^9966'                      :'DEVICE_COMPLICATION_ID'        ,
           '^9967'                      :'DEVICE_COMPLICATION_OTHER'     ,
           '^996[8-9]'                  :'TRANSPLANT_COMPLICATION_OTHER' ,
           '^997'                       :'COMPLICATION_OTHER'            ,
           '^99[8-9]'                   :'IATROGENIC_COMPLICATION'       ,
           '^V([0-2]|4)'                :'POTENTIAL_HEALTH_HAZARD'       ,
           '^V[5-9]'                    :'AFTERCARE_PROCEDURE'           ,
           '^V30'                       :'SINGLE_LIVEBORN'               ,
           '^V3[1-9]'                   :'MULTIPLE_LIVEBORN'             ,
           '^E(0|8[0-4])'               :'VEHICLE_ACCIDENT'          ,
           '^E8[5-6]'                   :'ACCIDENTAL_POISONING'          ,
           '^E(87|93)'                  :'MEDICAL_MISHAP'                ,
           '^E88'                       :'FALL'                          ,
           '^E(89|9([0-2]|[4-9]))'      :'INJURY_OTHER'                  }

In [1626]:
for i in range(1,4):
    for regx,dx in icd_map.items():
        X[f'DIAG_{i}_{dx}'] = X[f'DIAGNOSIS_{i}'].str.match(regx,na=False).astype(int)

In [1627]:
X.isna().sum().sum()

0

# Save

In [1630]:
one_level_columns = [c for c in X.columns if len(X[c].unique())<2]
print(one_level_columns)
X = X.drop(one_level_columns,axis=1)

['DIAG_1_VEHICLE_ACCIDENT', 'DIAG_1_ACCIDENTAL_POISONING', 'DIAG_1_MEDICAL_MISHAP', 'DIAG_1_FALL', 'DIAG_1_INJURY_OTHER', 'DIAG_2_SINGLE_LIVEBORN', 'DIAG_2_MULTIPLE_LIVEBORN', 'DIAG_3_SINGLE_LIVEBORN', 'DIAG_3_MULTIPLE_LIVEBORN']


In [1631]:
X.to_csv('X_y_plot.csv')
X.drop(['DIAGNOSIS','DIAGNOSIS_1','DIAGNOSIS_2','DIAGNOSIS_3'], axis=1).to_csv('X_y.csv')